# Proyecto Open Data I
## Radares, y su eficiencia en la CAM
### Recopilación, limpieza y almacenamiento de los datos
Este cuaderno pretende enseñar el proceso de limpieza de los datos relativos a los radares en la CAM

_Paula Gómez Lucas_

In [1]:
# Importar librerías
import os
import json
import pandas as pd
import numpy as np
import math
from pyproj import Proj, transform

A continuación, se muestra la clase que está compuesta de todos los métodos que se encargan de la limpieza y transformación de los datos

In [2]:
ds = {}
folders = ("datasets/actuacionesBomberos", "datasets/estaciones", "datasets/accidentalidad")
folders

('datasets/actuacionesBomberos',
 'datasets/estaciones',
 'datasets/accidentalidad')

In [3]:
for folder in folders:
    df = pd.DataFrame()
    filename = folder[9:]

    for file in os.listdir(folder):
        filepath = folder + "/" + file
        if folder == "datasets/actuacionesBomberos":
            df1 = pd.read_csv(filepath, sep=';', decimal=',', encoding='iso-8859-1', low_memory=False)
        else:
            df1 = pd.read_csv(filepath, sep=';', decimal=',', encoding='utf-8', low_memory=False)
        df = pd.concat([df, df1])

    ds[filename] = df

In [4]:
folder_path = "datasets"
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
csv_files

['DireccionesVigentes_20240204.csv',
 'iluminacion.csv',
 'padron22.csv',
 'radares.csv',
 'UbicacionEstacionesPermanentesSentidos.csv',
 'unidades_luminosas_m30.csv']

In [5]:
ds['nombres_estaciones'] = pd.read_csv('datasets/UbicacionEstacionesPermanentesSentidos.csv', sep=';', encoding='utf-8-sig', low_memory=False, decimal=",")


In [6]:
ds['direcciones'] = pd.read_csv('datasets/DireccionesVigentes_20240204.csv', sep=',', encoding='utf-8-sig', low_memory=False, decimal=".")
ds['iluminacion'] = pd.read_csv('datasets/iluminacion.csv', sep=',', encoding='utf-8-sig', low_memory=False, decimal=".")
ds['padron'] = pd.read_csv('datasets/padron22.csv', sep=',', encoding='iso-8859-1', low_memory=False)
ds['radares'] = pd.read_csv('datasets/radares.csv', sep=',', encoding='utf-8-sig', low_memory=False, decimal=".")
iluminacion = pd.read_csv('datasets/unidades_luminosas_m30.csv', sep=';', encoding='utf-8-sig', low_memory=False, decimal=',')
iluminacion.columns

Index(['tipo_bloque,C,50', 'TIPO,C,254', 'VIA_CLASE,C,254', 'VIA_PAR,C,254',
       'VIA_NOMBRE,C,254', 'CLASE_APP,C,254', 'NUMERO,N,10,0',
       'COD_NDP,N,10,0', 'DISTRITO,N,10,0', 'BARRIO,N,10,0', 'X_UTM,N,19,11',
       'Y_UTM,N,19,11'],
      dtype='object')

In [7]:
iluminacion.rename(columns={'tipo_bloque,C,50':'tipo_bloque', 'TIPO,C,254':'TIPO', 'VIA_CLASE,C,254':'VIA_CLASE', 'VIA_PAR,C,254':'VIA_PAR', 'VIA_NOMBRE,C,254':'VIA_NOMBRE', 'CLASE_APP,C,254':'CLASE_APP', 'NUMERO,N,10,0':'NUMERO', 'COD_NDP,N,10,0':'COD_NDP', 'DISTRITO,N,10,0':'DISTRITO', 'BARRIO,N,10,0':'BARRIO', 'X_UTM,N,19,11':'X_UTM', 'Y_UTM,N,19,11':'Y_UTM'}, inplace=True) 
print(iluminacion.columns)
print(ds['iluminacion'].columns)

Index(['tipo_bloque', 'TIPO', 'VIA_CLASE', 'VIA_PAR', 'VIA_NOMBRE',
       'CLASE_APP', 'NUMERO', 'COD_NDP', 'DISTRITO', 'BARRIO', 'X_UTM',
       'Y_UTM'],
      dtype='object')
Index(['tipo_bloque', 'TIPO', 'VIA_CLASE', 'VIA_PAR', 'VIA_NOMBRE',
       'CLASE_APP', 'NUMERO', 'COD_NDP', 'DISTRITO', 'BARRIO', 'X_UTM',
       'Y_UTM'],
      dtype='object')


In [8]:
ds['iluminacion'] = pd.concat([ds['iluminacion'], iluminacion])
ds['iluminacion']

,tipo_bloque,TIPO,VIA_CLASE,VIA_PAR,VIA_NOMBRE,CLASE_APP,NUMERO,COD_NDP,DISTRITO,BARRIO,X_UTM,Y_UTM
0,LBE002,DESCARGA,CALLE,DE LAS,ERAS,NUMERO,3,20040016,16,4,445648.4812,4480670.67
1,LBE002,DESCARGA,CALLE,DE LAS,ERAS,NUMERO,10,31024246,16,4,445610.6416,4480740.996
2,GLED001,LED,PLAZA,DE,MAR DEL PLATA,NUMERO,12,11119324,16,4,445680.0750,4480679.164
3,FFLED005,LED,CALLE,DEL,MAR AMARILLO,NUMERO,21,11119327,16,4,445669.5353,4480644.66
4,FFLED005,LED,CALLE,DEL,MAR AMARILLO,NUMERO,19,11119326,16,4,445651.1627,4480635.018
...,...,...,...,...,...,...,...,...,...,...,...,...
3609,B - Báculo.,SA - Sodio alta presión.,AUTOVIA,NaN,M-500,KM,99+370,99WT37ALUM01,9,1,437437.9738,"4476224,266,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
3610,B - Báculo.,SA - Sodio alta presión.,AUTOVIA,NaN,M-500,KM,99+420,99WT42ALUM01,9,1,437493.3321,"4476217,74,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
3611,B - Báculo.,SA - Sodio alta presión.,AUTOVIA,NaN,M-500,KM,99+460,99WT46ALUM01,9,1,437532.6933,"4476210,583,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
3612,B - Báculo.,SA - Sodio alta presión.,AUTOVIA,NaN,M-500,KM,99+500,99WT50ALUM01,9,1,437571.4231,"4476206,794,,,,,,,,,,,,,,,,,,,,,,,,,,,,"


Observamos que ambos datasets son un único dataframe ahora, teniendo todas las unidades luminosas en uno único. A continuación, eliminamos las variables auxiliares que hemos utilizado.

In [9]:
del iluminacion
del df
del df1

Limpiamos los datos eliminando las columnas autogeneradas con NaNs, renombramos las columnas para que sean uniformes (minúsculas y con barra bajas), eliminamos las filas de NaNs, eliminamos las filas duplicadas.

In [10]:
columna_borrar = "Unnamed"
for df in ds:
    for j in ds[df].columns:
        if columna_borrar in j:
            while j in ds[df].columns:
                ds[df].drop(j, axis=1, inplace=True)
                ds[df].dropna(how='all', axis=0, inplace=True)

    ds[df].rename(columns = lambda x: x.strip().lower().replace(' ', '_'), inplace=True)
    ds[df] = ds[df].map(lambda x: x.strip() if isinstance(x, str) else x)
    ds[df].dropna(how='all', axis=0, inplace=True)
    ds[df].drop_duplicates(inplace=True)
    ds[df] = ds[df].loc[:, ~ds[df].columns.duplicated()]

Para confirmar que los datos se van limpiando bien, utilizaremos el método get_info para ver cuántos datos en cada columna quedan nulos, así como usamos el método info() de pandas para ver un resumen de todas las columnas y comprobamos también que todo el formateo de las columnas se ha realizado sin problema.

In [11]:
def get_info(filename):
    print(ds[filename].isnull().sum())
    print(ds[filename].info())

El siguiente paso es el análisis de los datasets columna a columna para revisar qué método usar para rellenar los datos faltantes dependiendo de cada atributo.  

### Sustitución de valores faltantes
El dataset estaciones no tienen datos faltantes, por lo que sólo nos queda trabajar con los otros 8 datasets, que podemos observar aquí:

In [12]:
ds.keys()

dict_keys(['actuacionesBomberos', 'estaciones', 'accidentalidad', 'nombres_estaciones', 'direcciones', 'iluminacion', 'padron', 'radares'])

#### Actuaciones de bomberos

In [13]:
get_info('actuacionesBomberos')

año                         1043
mes                            1
distrito                       1
fuegos                         1
daños_en_construccion       1043
salvamentos_y_rescates         1
daños_por_agua              1043
incidentes_diversos            1
salidas_sin_intervencion       1
servicios_varios               1
total                          1
ï»¿aão                      259
daãos_en_construccion       260
daãos_por_agua              260
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 1302 entries, 0 to 263
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   año                       259 non-null    float64
 1   mes                       1301 non-null   object 
 2   distrito                  1301 non-null   object 
 3   fuegos                    1301 non-null   float64
 4   daños_en_construccion     259 non-null    float64
 5   salvamentos_y_rescates    1301 non

In [14]:
str(ds['actuacionesBomberos'].columns)

"Index(['año', 'mes', 'distrito', 'fuegos', 'daños_en_construccion',\n       'salvamentos_y_rescates', 'daños_por_agua', 'incidentes_diversos',\n       'salidas_sin_intervencion', 'servicios_varios', 'total', 'ï»¿aã\x91o',\n       'daã\x91os_en_construccion', 'daã\x91os_por_agua'],\n      dtype='object')"

Vemos que las columnas de daños por agua, construcción y año, se han disgregado en alguno de los csv iniciales por formato, por lo que salen como faltantes valores que realmente no lo son. Una solución es sustituir los faltantes por 0 y sumar los valores de las columnas a mergear porque así se concatenan los valores sin despreciar su tipo.

In [15]:
ds['actuacionesBomberos']['daños_por_agua'] = ds['actuacionesBomberos']['daños_por_agua'].fillna(0)
ds['actuacionesBomberos']['daños_en_construccion'] = ds['actuacionesBomberos']['daños_en_construccion'].fillna(0)
ds['actuacionesBomberos']['año'] = ds['actuacionesBomberos']['año'].fillna(0)
ds['actuacionesBomberos']['ï»¿aã\x91o'] = ds['actuacionesBomberos']['ï»¿aã\x91o'].fillna(0)
ds['actuacionesBomberos']['daã\x91os_en_construccion'] = ds['actuacionesBomberos']['daã\x91os_en_construccion'].fillna(0)
ds['actuacionesBomberos']['daã\x91os_por_agua'] = ds['actuacionesBomberos']['daã\x91os_por_agua'].fillna(0)


In [16]:
get_info('actuacionesBomberos')
ds['actuacionesBomberos']

año                         0
mes                         1
distrito                    1
fuegos                      1
daños_en_construccion       0
salvamentos_y_rescates      1
daños_por_agua              0
incidentes_diversos         1
salidas_sin_intervencion    1
servicios_varios            1
total                       1
ï»¿aão                     0
daãos_en_construccion      0
daãos_por_agua             0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 1302 entries, 0 to 263
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   año                       1302 non-null   float64
 1   mes                       1301 non-null   object 
 2   distrito                  1301 non-null   object 
 3   fuegos                    1301 non-null   float64
 4   daños_en_construccion     1302 non-null   float64
 5   salvamentos_y_rescates    1301 non-null   float64
 6   daños_por_agua       

,año,mes,distrito,fuegos,daños_en_construccion,salvamentos_y_rescates,daños_por_agua,incidentes_diversos,salidas_sin_intervencion,servicios_varios,total,ï»¿aão,daãos_en_construccion,daãos_por_agua
0,2019.0,Enero,CENTRO,27.0,20.0,35.0,21.0,41.0,16.0,10.0,170.0,0.0,0.0,0.0
1,2019.0,Enero,ARGANZUELA,16.0,9.0,13.0,8.0,24.0,8.0,3.0,81.0,0.0,0.0,0.0
2,2019.0,Enero,RETIRO,7.0,4.0,7.0,4.0,18.0,6.0,4.0,50.0,0.0,0.0,0.0
3,2019.0,Enero,SALAMANCA,21.0,15.0,16.0,14.0,31.0,17.0,7.0,121.0,0.0,0.0,0.0
4,2019.0,Enero,CHAMARTIN,12.0,7.0,21.0,5.0,25.0,6.0,5.0,81.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,0.0,enero,VILLA DE VALLECAS,15.0,0.0,19.0,0.0,7.0,22.0,2.0,77.0,2023.0,7.0,5.0
260,0.0,enero,VICALVARO,9.0,0.0,8.0,0.0,8.0,6.0,2.0,41.0,2023.0,5.0,3.0
261,0.0,enero,SAN BLAS,16.0,0.0,25.0,0.0,10.0,20.0,1.0,79.0,2023.0,3.0,4.0
262,0.0,enero,BARAJAS,1.0,0.0,4.0,0.0,15.0,13.0,8.0,45.0,2023.0,2.0,2.0


In [17]:
ds['actuacionesBomberos']['daños_por_agua'] = ds['actuacionesBomberos']['daños_por_agua'] + ds['actuacionesBomberos']['daã\x91os_por_agua']
ds['actuacionesBomberos']['daños_en_construccion'] = ds['actuacionesBomberos']['daños_en_construccion'] + ds['actuacionesBomberos']['daã\x91os_en_construccion']
ds['actuacionesBomberos']['año'] = ds['actuacionesBomberos']['año'] + ds['actuacionesBomberos']['ï»¿aã\x91o']

In [18]:
ds['actuacionesBomberos'].drop('ï»¿aã\x91o', axis=1, inplace=True)
ds['actuacionesBomberos'].drop('daã\x91os_por_agua', axis=1, inplace=True)
ds['actuacionesBomberos'].drop('daã\x91os_en_construccion', axis=1, inplace=True)

In [19]:
get_info('actuacionesBomberos')
ds['actuacionesBomberos']

año                         0
mes                         1
distrito                    1
fuegos                      1
daños_en_construccion       0
salvamentos_y_rescates      1
daños_por_agua              0
incidentes_diversos         1
salidas_sin_intervencion    1
servicios_varios            1
total                       1
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 1302 entries, 0 to 263
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   año                       1302 non-null   float64
 1   mes                       1301 non-null   object 
 2   distrito                  1301 non-null   object 
 3   fuegos                    1301 non-null   float64
 4   daños_en_construccion     1302 non-null   float64
 5   salvamentos_y_rescates    1301 non-null   float64
 6   daños_por_agua            1302 non-null   float64
 7   incidentes_diversos       1301 non-null   float64
 8   s

,año,mes,distrito,fuegos,daños_en_construccion,salvamentos_y_rescates,daños_por_agua,incidentes_diversos,salidas_sin_intervencion,servicios_varios,total
0,2019.0,Enero,CENTRO,27.0,20.0,35.0,21.0,41.0,16.0,10.0,170.0
1,2019.0,Enero,ARGANZUELA,16.0,9.0,13.0,8.0,24.0,8.0,3.0,81.0
2,2019.0,Enero,RETIRO,7.0,4.0,7.0,4.0,18.0,6.0,4.0,50.0
3,2019.0,Enero,SALAMANCA,21.0,15.0,16.0,14.0,31.0,17.0,7.0,121.0
4,2019.0,Enero,CHAMARTIN,12.0,7.0,21.0,5.0,25.0,6.0,5.0,81.0
...,...,...,...,...,...,...,...,...,...,...,...
259,2023.0,enero,VILLA DE VALLECAS,15.0,7.0,19.0,5.0,7.0,22.0,2.0,77.0
260,2023.0,enero,VICALVARO,9.0,5.0,8.0,3.0,8.0,6.0,2.0,41.0
261,2023.0,enero,SAN BLAS,16.0,3.0,25.0,4.0,10.0,20.0,1.0,79.0
262,2023.0,enero,BARAJAS,1.0,2.0,4.0,2.0,15.0,13.0,8.0,45.0


Debido a que el MV que encontramos está en todas las columnas menos en las que sustituimos por 0 para realizar la limpieza, parece que todos los valores realmente eran faltantes inicialmente: 

In [20]:
ds['actuacionesBomberos'][ds['actuacionesBomberos'].isna().any(axis=1)]

,año,mes,distrito,fuegos,daños_en_construccion,salvamentos_y_rescates,daños_por_agua,incidentes_diversos,salidas_sin_intervencion,servicios_varios,total
257,2020.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN


In [21]:
ds['actuacionesBomberos'].drop(257, axis=0, inplace=True)

In [22]:
ds['actuacionesBomberos']['año'] = ds['actuacionesBomberos']['año'].astype('int64')
ds['actuacionesBomberos']['fuegos'] = ds['actuacionesBomberos']['fuegos'].astype('int64')
ds['actuacionesBomberos']['daños_en_construccion'] = ds['actuacionesBomberos']['daños_en_construccion'].astype('int64')
ds['actuacionesBomberos']['salvamentos_y_rescates'] = ds['actuacionesBomberos']['salvamentos_y_rescates'].astype('int64')
ds['actuacionesBomberos']['daños_por_agua'] = ds['actuacionesBomberos']['daños_por_agua'].astype('int64')
ds['actuacionesBomberos']['incidentes_diversos'] = ds['actuacionesBomberos']['incidentes_diversos'].astype('int64')
ds['actuacionesBomberos']['salidas_sin_intervencion'] = ds['actuacionesBomberos']['salidas_sin_intervencion'].astype('int64')
ds['actuacionesBomberos']['servicios_varios'] = ds['actuacionesBomberos']['servicios_varios'].astype('int64')
ds['actuacionesBomberos']['total'] = ds['actuacionesBomberos']['total'].astype('int64')

In [23]:
ds['actuacionesBomberos']['mes'] = ds['actuacionesBomberos']['mes'].str.lower()
meses = {
    1:'enero',
    2:'febrero',
    3:'marzo',
    4:'abril',
    5:'mayo',
    6:'junio',
    7:'julio',
    8:'agosto',
    9:'septiembre',
    10:'octubre',
    11:'noviembre',
    12:'diciembre'
}

ds['actuacionesBomberos']['idMes'] = ds['actuacionesBomberos']['mes'].map({v: k for k, v in meses.items()})

In [24]:
get_info('actuacionesBomberos')
ds['actuacionesBomberos']

año                         0
mes                         0
distrito                    0
fuegos                      0
daños_en_construccion       0
salvamentos_y_rescates      0
daños_por_agua              0
incidentes_diversos         0
salidas_sin_intervencion    0
servicios_varios            0
total                       0
idMes                       0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 1297 entries, 0 to 263
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   año                       1297 non-null   int64 
 1   mes                       1297 non-null   object
 2   distrito                  1297 non-null   object
 3   fuegos                    1297 non-null   int64 
 4   daños_en_construccion     1297 non-null   int64 
 5   salvamentos_y_rescates    1297 non-null   int64 
 6   daños_por_agua            1297 non-null   int64 
 7   incidentes_diversos       1297 non-

,año,mes,distrito,fuegos,daños_en_construccion,salvamentos_y_rescates,daños_por_agua,incidentes_diversos,salidas_sin_intervencion,servicios_varios,total,idMes
0,2019,enero,CENTRO,27,20,35,21,41,16,10,170,1
1,2019,enero,ARGANZUELA,16,9,13,8,24,8,3,81,1
2,2019,enero,RETIRO,7,4,7,4,18,6,4,50,1
3,2019,enero,SALAMANCA,21,15,16,14,31,17,7,121,1
4,2019,enero,CHAMARTIN,12,7,21,5,25,6,5,81,1
...,...,...,...,...,...,...,...,...,...,...,...,...
259,2023,enero,VILLA DE VALLECAS,15,7,19,5,7,22,2,77,1
260,2023,enero,VICALVARO,9,5,8,3,8,6,2,41,1
261,2023,enero,SAN BLAS,16,3,25,4,10,20,1,79,1
262,2023,enero,BARAJAS,1,2,4,2,15,13,8,45,1


In [25]:
ds['actuacionesBomberos']['distrito'] = ds['actuacionesBomberos']['distrito'].str.replace('FUERA DEL TÃ\x89RMINO MUNICIPAL', 'FUERA DEL TERMINO MUNICIPAL')
ds['actuacionesBomberos']['distrito'] = ds['actuacionesBomberos']['distrito'].str.replace('SIN DISTRITO', 'FUERA DEL TERMINO MUNICIPAL')
ds['actuacionesBomberos']['distrito'] = ds['actuacionesBomberos']['distrito'].str.replace('FUERA TERMINO MUNICIPAL', 'FUERA DEL TERMINO MUNICIPAL')

Para unificar el lenguaje entre datasets, vemos cómo se califican aquí y en otros a los distritos:

In [26]:
unicos = ds['actuacionesBomberos']['distrito'].unique()
unicos.sort()
print(unicos)

['ARGANZUELA' 'BARAJAS' 'CARABANCHEL' 'CENTRO' 'CHAMARTIN' 'CHAMBERI'
 'CIUDAD LINEAL' 'FUENCARRAL' 'FUERA DEL TERMINO MUNICIPAL' 'HORTALEZA'
 'LATINA' 'MONCLOA' 'MORATALAZ' 'PUENTE VALLECAS' 'RETIRO' 'SALAMANCA'
 'SAN BLAS' 'TETUAN' 'USERA' 'VICALVARO' 'VILLA DE VALLECAS' 'VILLAVERDE']


In [27]:
unicos2 = ds['accidentalidad']['distrito'].unique()
print(unicos2)

['CENTRO' 'CARABANCHEL' 'LATINA' 'USERA' 'MONCLOA-ARAVACA' 'MORATALAZ'
 'SALAMANCA' 'VILLA DE VALLECAS' 'VILLAVERDE' 'CHAMBERÍ' 'CHAMARTÍN'
 'HORTALEZA' 'CIUDAD LINEAL' 'RETIRO' 'FUENCARRAL-EL PARDO' 'VICÁLVARO'
 'PUENTE DE VALLECAS' 'BARAJAS' 'ARGANZUELA' 'TETUÁN'
 'SAN BLAS-CANILLEJAS' nan]


In [28]:
ds['actuacionesBomberos'].to_csv('limpios/bomberos.csv', index=False)

#### Accidentalidad

In [29]:
get_info('accidentalidad')
ds['accidentalidad']

num_expediente               0
fecha                        0
hora                         0
localizacion                 0
numero                       6
cod_distrito                 6
distrito                     6
tipo_accidente               5
estado_meteorológico     20120
tipo_vehiculo              816
tipo_persona                 3
rango_edad                   0
sexo                         0
cod_lesividad            85278
lesividad                85278
coordenada_x_utm             3
coordenada_y_utm             3
positiva_alcohol           718
positiva_droga          195001
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 195610 entries, 0 to 31157
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   num_expediente        195610 non-null  object 
 1   fecha                 195610 non-null  object 
 2   hora                  195610 non-null  object 
 3   localizacion          195610

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
0,2018S017842,04/02/2019,9:10:00,"CALL. ALBERTO AGUILERA, 1",1,1.0,CENTRO,Colisión lateral,Despejado,Motocicleta > 125cc,Conductor,De 45 a 49 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,"440068,049","4475679,17",N,NaN
1,2018S017842,04/02/2019,9:10:00,"CALL. ALBERTO AGUILERA, 1",1,1.0,CENTRO,Colisión lateral,Despejado,Turismo,Conductor,De 30 a 34 años,Mujer,7.0,Asistencia sanitaria sólo en el lugar del acci...,"440068,049","4475679,17",N,NaN
2,2019S000001,01/01/2019,3:45:00,PASEO. SANTA MARIA DE LA CABEZA / PLAZA. ELIPTICA,168,11.0,CARABANCHEL,Alcance,NaN,Furgoneta,Conductor,De 40 a 44 años,Hombre,NaN,NaN,"439139,603","4470836,854",S,NaN
3,2019S000001,01/01/2019,3:45:00,PASEO. SANTA MARIA DE LA CABEZA / PLAZA. ELIPTICA,168,11.0,CARABANCHEL,Alcance,NaN,Turismo,Conductor,De 40 a 44 años,Mujer,NaN,NaN,"439139,603","4470836,854",N,NaN
4,2019S000001,01/01/2019,3:45:00,PASEO. SANTA MARIA DE LA CABEZA / PLAZA. ELIPTICA,168,11.0,CARABANCHEL,Alcance,NaN,Turismo,Conductor,De 45 a 49 años,Mujer,NaN,NaN,"439139,603","4470836,854",N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31153,2023S028337,31/08/2023,23:20:00,"CALL. GOLFO DE SALONICA, 12",12,16.0,HORTALEZA,Otro,Despejado,Turismo,Pasajero,De 55 a 59 años,Mujer,14.0,Sin asistencia sanitaria,443580.365,4480912.376,N,NaN
31154,2023S028341,31/08/2023,22:30:00,CALLE SAN SERAPIO,1,12.0,USERA,Alcance,Despejado,Motocicleta hasta 125cc,Conductor,De 30 a 34 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,440795.550,4471044.789,N,NaN
31155,2023S028341,31/08/2023,22:30:00,CALLE SAN SERAPIO,1,12.0,USERA,Alcance,Despejado,Turismo,Conductor,De 35 a 39 años,Hombre,14.0,Sin asistencia sanitaria,440795.550,4471044.789,N,NaN
31156,2023S028352,31/08/2023,14:40:00,"AUTOV. M-30, 00NC06",00NC06,5.0,CHAMARTÍN,Caída,Despejado,Motocicleta hasta 125cc,Conductor,De 40 a 44 años,Hombre,1.0,Atención en urgencias sin posterior ingreso,442825.500,4481003.193,N,NaN


In [30]:
lesividadDict = ds['accidentalidad'].set_index('cod_lesividad')['lesividad'].to_dict()
lesividadDict = {int(k): v for k, v in lesividadDict.items() if not math.isnan(k)}
lesividadDict[14] = 'Sin asistencia sanitaria'
lesividadDict


{7: 'Asistencia sanitaria sólo en el lugar del accidente',
 2: 'Ingreso inferior o igual a 24 horas',
 14: 'Sin asistencia sanitaria',
 5: 'Asistencia sanitaria ambulatoria con posterioridad',
 3: 'Ingreso superior a 24 horas',
 1: 'Atención en urgencias sin posterior ingreso',
 6: 'Asistencia sanitaria inmediata en centro de salud o mutua',
 4: 'Fallecido 24 horas',
 77: 'Se desconoce'}

Hay algunos datos faltantes que tiene sentido que lo sean, y podemos sustituir por un buzzword de algún tipo que nos haga saber que se trate de esto, como lo es que en un accidente en el que no se han producido lesiones, la lesividad sea nula y tampoco haya código de la misma, y como ambas cifras coinciden, es lógico pensar que se trata de las mismas situaciones. Podemos sustituir entonces todos los NaNs de lesividad faltantes por 'Sin asistencia sanitaria' y el código por 14, ya que si no hay registro, es que no se realizó dicha asistencia.

In [31]:
ds['accidentalidad']['cod_lesividad'] = ds['accidentalidad']['cod_lesividad'].fillna(14.0)
ds['accidentalidad']['lesividad'] = ds['accidentalidad']['lesividad'].fillna(ds['accidentalidad']['cod_lesividad'].map(lesividadDict))

In [32]:
ds['accidentalidad']['lesividad'] = np.where(ds['accidentalidad']['cod_lesividad'] == 14,
                                             ds['accidentalidad']['lesividad'].fillna('Sin asistencia sanitaria'),
                                             ds['accidentalidad']['lesividad'])


Renombramos los valores de distritos para mayor uniformidad entre datasets

In [33]:
ds['accidentalidad']['distrito'] = ds['accidentalidad']['distrito'].str.replace('TETUÁN', 'TETUAN')
ds['accidentalidad']['distrito'] = ds['accidentalidad']['distrito'].str.replace('FUENCARRAL-EL PARDO', 'FUENCARRAL')
ds['accidentalidad']['distrito'] = ds['accidentalidad']['distrito'].str.replace('CHAMBERÍ', 'CHAMBERI')
ds['accidentalidad']['distrito'] = ds['accidentalidad']['distrito'].str.replace('CHAMARTÍN', 'CHAMARTIN')
ds['accidentalidad']['distrito'] = ds['accidentalidad']['distrito'].str.replace('MONCLOA-ARAVACA', 'MONCLOA')
ds['accidentalidad']['distrito'] = ds['accidentalidad']['distrito'].str.replace('VICÁLVARO', 'VICALVARO')
ds['accidentalidad']['distrito'] = ds['accidentalidad']['distrito'].str.replace('SAN BLAS-CANILLEJAS', 'SAN BLAS')
ds['accidentalidad']['distrito'] = ds['accidentalidad']['distrito'].str.replace('PUENTE DE VALLECAS', 'PUENTE VALLECAS')

Ocurre una situación similar en distrito y código distrito

In [34]:
distritoDict = ds['accidentalidad'].set_index('cod_distrito')['distrito'].to_dict()
distritoDict = {int(k): v for k, v in distritoDict.items() if not math.isnan(k)}
distritoDict

{1: 'CENTRO',
 11: 'CARABANCHEL',
 10: 'LATINA',
 12: 'USERA',
 9: 'MONCLOA',
 14: 'MORATALAZ',
 4: 'SALAMANCA',
 18: 'VILLA DE VALLECAS',
 17: 'VILLAVERDE',
 7: 'CHAMBERI',
 5: 'CHAMARTIN',
 16: 'HORTALEZA',
 15: 'CIUDAD LINEAL',
 3: 'RETIRO',
 8: 'FUENCARRAL',
 19: 'VICALVARO',
 13: 'PUENTE VALLECAS',
 21: 'BARAJAS',
 2: 'ARGANZUELA',
 6: 'TETUAN',
 20: 'SAN BLAS'}

In [35]:
distritoDict[22] = 'FUERA DEL TERMINO MUNICIPAL'

In [36]:
ds['accidentalidad']['distrito'] = ds['accidentalidad']['distrito'].fillna(ds['accidentalidad']['cod_distrito'].map(distritoDict))

In [37]:
get_info('accidentalidad')
# Get all info on accidentalidad

num_expediente               0
fecha                        0
hora                         0
localizacion                 0
numero                       6
cod_distrito                 6
distrito                     6
tipo_accidente               5
estado_meteorológico     20120
tipo_vehiculo              816
tipo_persona                 3
rango_edad                   0
sexo                         0
cod_lesividad                0
lesividad                    0
coordenada_x_utm             3
coordenada_y_utm             3
positiva_alcohol           718
positiva_droga          195001
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 195610 entries, 0 to 31157
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   num_expediente        195610 non-null  object 
 1   fecha                 195610 non-null  object 
 2   hora                  195610 non-null  object 
 3   localizacion          195610

Por otro lado, positivo en droga tiene valor sólo si daba positivo, por lo que rellenar los valores faltantes con 0 es lo más lógico (siendo 0 negativo en droga). 

In [38]:
ds['accidentalidad']['positiva_droga'] = ds['accidentalidad']['positiva_droga'].fillna(0)

Observemos primero los valores que faltan y que vamos a decidir rellenar muy probablemente de manera manual, ya que son menos de 10 valores por atributo que faltan. En concreto, las columnas referentes al distrito, a la localizacion (el número y las coordenadas), y el tipo de accidente y persona.

In [39]:
ds['accidentalidad'][ds['accidentalidad'][['distrito', 'cod_distrito', 'tipo_accidente', 'tipo_persona', 'coordenada_x_utm', 'coordenada_y_utm']].isna().any(axis=1)]

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
27635,2020S016821,23/11/2020,7:45:00,"AUTOV. M-23, 0 (0.8 ENTRADA)",NaN,NaN,NaN,Colisión lateral,Despejado,Turismo,Conductor,De 50 a 54 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,"444975,42","4474103,079",N,0.0
27636,2020S016821,23/11/2020,7:45:00,"AUTOV. M-23, 0 (0.8 ENTRADA)",NaN,NaN,NaN,Colisión lateral,Despejado,Turismo,Conductor,De 50 a 54 años,Mujer,14.0,Sin asistencia sanitaria,"444975,42","4474103,079",N,0.0
32430,2020S019647,18/11/2020,10:57:00,"CALL. LOPE DE HARO, 8",8,6.0,TETUAN,Colisión fronto-lateral,Despejado,Camión rígido,Conductor,De 40 a 44 años,Hombre,14.0,Sin asistencia sanitaria,NaN,NaN,N,0.0
32431,2020S019647,18/11/2020,10:57:00,"CALL. LOPE DE HARO, 8",8,6.0,TETUAN,Colisión fronto-lateral,Despejado,Furgoneta,Conductor,De 45 a 49 años,Hombre,14.0,Sin asistencia sanitaria,NaN,NaN,N,0.0
25744,2021S015933,11/09/2021,19:40:00,AUTOV. M-500 / AUTOV. M-30,NaN,NaN,NaN,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,14.0,Sin asistencia sanitaria,"437390,155","4476258,031",N,0.0
25745,2021S015933,11/09/2021,19:40:00,AUTOV. M-500 / AUTOV. M-30,NaN,NaN,NaN,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 55 a 59 años,Hombre,14.0,Sin asistencia sanitaria,"437390,155","4476258,031",N,0.0
25746,2021S015933,11/09/2021,19:40:00,AUTOV. M-500 / AUTOV. M-30,NaN,NaN,NaN,Colisión fronto-lateral,Despejado,Turismo,Pasajero,De 55 a 59 años,Mujer,14.0,Sin asistencia sanitaria,"437390,155","4476258,031",N,0.0
36915,2021S022402,27/11/2021,2:10:00,PASEO. PRADO / PLAZA. CANOVAS DEL CASTILLO,36,1.0,CENTRO,NaN,NaN,Turismo,Conductor,De 55 a 59 años,Hombre,14.0,Sin asistencia sanitaria,"441066,084","4474148,543",N,0.0
36916,2021S022402,27/11/2021,2:10:00,PASEO. PRADO / PLAZA. CANOVAS DEL CASTILLO,36,1.0,CENTRO,NaN,NaN,Turismo,Pasajero,De 35 a 39 años,Hombre,14.0,Sin asistencia sanitaria,"441066,084","4474148,543",N,0.0
36917,2021S022402,27/11/2021,2:10:00,PASEO. PRADO / PLAZA. CANOVAS DEL CASTILLO,36,1.0,CENTRO,NaN,NaN,VMU eléctrico,Conductor,De 25 a 29 años,Hombre,2.0,Ingreso inferior o igual a 24 horas,"441066,084","4474148,543",S,0.0


In [40]:
ds['accidentalidad'].at[10110, 'cod_distrito'] = 21
ds['accidentalidad'].at[27635, 'cod_distrito'] = 15
ds['accidentalidad'].at[27636, 'cod_distrito'] = 15
ds['accidentalidad'].at[25744, 'cod_distrito'] = 9
ds['accidentalidad'].at[25745, 'cod_distrito'] = 9
ds['accidentalidad'].at[25746, 'cod_distrito'] = 9
ds['accidentalidad']['distrito'] = ds['accidentalidad']['distrito'].fillna(ds['accidentalidad']['cod_distrito'].map(distritoDict))

In [41]:
ds['accidentalidad']['numero'] = ds['accidentalidad']['numero'].fillna('')

Para el tipo de persona, usamos rango de edad. Si la persona es menor de edad, no era conductor, y si el rango de edad es desconocido, el tipo de persona es conductor si hay una única incidencia.

In [42]:
ds['accidentalidad'][(ds['accidentalidad']['tipo_persona'].isna()) & 
                              (ds['accidentalidad'].duplicated(subset=['fecha', 'hora', 'coordenada_x_utm', 'coordenada_y_utm'], keep=False))]

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
37104,2021S022518,27/11/2021,22:25:00,"AVDA. HISPANIDAD, 34",34,21.0,BARAJAS,Alcance,Despejado,Turismo,NaN,Desconocido,Desconocido,14.0,Sin asistencia sanitaria,"451305,806","4479449,657",N,0.0
40495,2021S024425,21/12/2021,16:30:00,AVENIDA MANUEL FRAGA IRIBARNE NUMERO 2,35,21.0,BARAJAS,Alcance,Despejado,Turismo,NaN,Menor de 5 años,Hombre,14.0,Sin asistencia sanitaria,"448419,744","4481821,54",N,0.0
41575,2021S024983,30/12/2021,17:50:00,CALL. VALDETORRES DE JARAMA / CALL. ANGEL LUIS...,2,16.0,HORTALEZA,Colisión fronto-lateral,Despejado,Motocicleta hasta 125cc,NaN,Desconocido,Desconocido,14.0,Sin asistencia sanitaria,"445289,712","4480250,615",N,0.0


In [43]:
ds['accidentalidad'].at[40495, 'tipo_persona'] = 'Pasajero'
ds['accidentalidad'].at[41575 , 'tipo_persona'] = 'Conductor'
ds['accidentalidad'].at[37104, 'tipo_persona'] = 'Conductor'

Para las coordenadas, las buscamos en internet y las rellenamos manualmente

In [44]:
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2020S019647', ['coordenada_x_utm', 'coordenada_y_utm']] = [440444, 4478766]
ds['accidentalidad'].at[41231, 'coordenada_x_utm'] = 445058
ds['accidentalidad'].at[41231, 'coordenada_y_utm'] = 4475358
ds['accidentalidad'].at[26525, 'coordenada_x_utm'] = 440766
ds['accidentalidad'].at[26525, 'coordenada_y_utm'] = 4468560

Confirmamos que realmente no queda ningun NaN en las coordenadas

In [45]:
df = ds['accidentalidad'][(ds['accidentalidad']['coordenada_x_utm'] == '#¡VALOR!') | (ds['accidentalidad']['coordenada_y_utm'] == '#¡VALOR!')]

Como sí hay, mostremos los numeros de expediente con la localizacion y el distrito, para poder buscar las coordenadas manualmente

In [46]:
df[['localizacion', 'distrito', 'num_expediente', 'tipo_accidente']].drop_duplicates(subset='num_expediente')

,localizacion,distrito,num_expediente,tipo_accidente
29308,"AUTOV. M-30, +03200E",FUENCARRAL,2019S026997,Alcance
17953,"CALL. POLVORANCA, 13",CARABANCHEL,2020S011111,Choque contra obstáculo fijo
20493,AVDA. ALBUFERA / AVDA. PABLO NERUDA,PUENTE VALLECAS,2020S012560,Colisión fronto-lateral
30817,RONDA. SUR / CALL. MARTOS,PUENTE VALLECAS,2021S018891,Alcance
37939,"CALL. REY FRANCISCO, 28",MONCLOA,2021S022950,Colisión múltiple
39443,"AUTOV. M-23, KM 0,700",MORATALAZ,2021S023850,Colisión múltiple
41425,URB. MANZANA DE AZCA / CALL. AGUSTIN DE BETANC...,TETUAN,2021S024915,Choque contra obstáculo fijo
41524,CALL. GENERAL RICARDOS / CALL. BATALLA DE TORR...,CARABANCHEL,2021S024959,Alcance
41614,"CALL. ARROYO DE LA MEDIA LEGUA, 72",MORATALAZ,2021S024999,Atropello a persona
802,"CALL. GENERAL RICARDOS, 15",CARABANCHEL,2022S000436,Caída


In [47]:
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2019S026997', ['coordenada_x_utm', 'coordenada_y_utm']] = [440339, 4481526]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2020S011111', ['coordenada_x_utm', 'coordenada_y_utm']] = [436021, 4469424]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2020S012560', ['coordenada_x_utm', 'coordenada_y_utm']] = [445353, 4471146]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2021S018891', ['coordenada_x_utm', 'coordenada_y_utm']] = [443848, 4469603]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2021S022950', ['coordenada_x_utm', 'coordenada_y_utm']] = [439091, 4475312]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2021S023850', ['coordenada_x_utm', 'coordenada_y_utm']] = [444822, 4474145]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2021S024915', ['coordenada_x_utm', 'coordenada_y_utm']] = [441103, 4477883]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2021S024959', ['coordenada_x_utm', 'coordenada_y_utm']] = [437104, 4470605]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2021S024999', ['coordenada_x_utm', 'coordenada_y_utm']] = [444742, 4473996]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2022S000436', ['coordenada_x_utm', 'coordenada_y_utm']] = [439117, 4472055]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2022S036787', ['coordenada_x_utm', 'coordenada_y_utm']] = [442816, 4475162]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2022S037553', ['coordenada_x_utm', 'coordenada_y_utm']] = [444195, 4471576]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2022S039457', ['coordenada_x_utm', 'coordenada_y_utm']] = [438802, 4468600]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2022S040326', ['coordenada_x_utm', 'coordenada_y_utm']] = [443181, 4472205]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2022S041278', ['coordenada_x_utm', 'coordenada_y_utm']] = [440884, 4477570]


In [48]:
ds['accidentalidad']['tipo_accidente'] = ds['accidentalidad']['tipo_accidente'].fillna(ds['accidentalidad']['tipo_accidente'].mode()[0])

In [49]:
tipos_accidente = ds['accidentalidad']['tipo_accidente'].unique()
for tipo in tipos_accidente:
    print(tipo)

Colisión lateral
Alcance
Choque contra obstáculo fijo
Colisión fronto-lateral
Caída
Colisión frontal
Otro
Atropello a persona
Colisión múltiple
Vuelco
Atropello a animal
Solo salida de la vía
Despeñamiento


Según la documentación de noviembre de 2023, redefinimos los tipos, siendo 
- Colisión doble: Accidente de tráfico ocurrido entre dos vehículos en movimiento, (colisión frontal, fronto lateral, lateral)
    - Aquí incluimos los antiguos tipos Colisión lateral, Colisión fronto-lateral, Colisión frontal
- Otras causas: Recoge los accidentes por atropello a animal, despeñamiento, salida de la vía, y otros
    - Incluimos los antiguos tipos Otro, Despeñamiento, Atropello a animal, y Solo salida de la vía
- Choque contra obstáculo o elemento de la vía: Accidente ocurrido entre un vehículo en movimiento con conductor y un objeto inmóvil que ocupa la vía o zona apartada de la misma, ya sea vehículo estacionado, árbol, farola, etc. 
    - Este no más es un cambio de nombre de Choque contra obstáculo fijo

Los tipos que se mantienen son:

- Alcance: Accidente que se produce cuando un vehículo circulando o detenido por las 
circunstancias del tráfico es golpeado en su parte posterior por otro vehículo.
- Caída: Se agrupan todas las caídas relacionadas con el desarrollo y las circunstancias del 
tráfico, (motocicleta, ciclomotor, bicicleta, viajero bus, etc.)
- Atropello a persona: Accidente ocurrido ente un vehículo y un peatón que ocupa la calzada 
o que transita por aceras, refugios, paseos o zonas de la vía pública no destinada a la
circulación de vehículos
- Vuelco: Accidente sufrido por un vehículo con más de dos ruedas y que por alguna 
circunstancia sus neumáticos pierden el contacto con la calzada quedando apoyado sobre 
un costado o sobre el techo.
- Colisión múltiple: Accidente de tráfico ocurrido entre más de dos vehículos en movimiento.


In [50]:
ds['accidentalidad']['tipo_accidente'] = pd.Categorical(ds['accidentalidad']['tipo_accidente'])
ds['accidentalidad']['cod_accidente'] = ds['accidentalidad']['tipo_accidente'].cat.codes
accidenteDict = ds['accidentalidad'].set_index('cod_accidente')['tipo_accidente'].to_dict()
accidenteDict

{7: 'Colisión lateral',
 0: 'Alcance',
 4: 'Choque contra obstáculo fijo',
 6: 'Colisión fronto-lateral',
 3: 'Caída',
 5: 'Colisión frontal',
 10: 'Otro',
 2: 'Atropello a persona',
 8: 'Colisión múltiple',
 12: 'Vuelco',
 1: 'Atropello a animal',
 11: 'Solo salida de la vía',
 9: 'Despeñamiento'}

In [51]:
ds['accidentalidad']['tipo_accidente'] = ds['accidentalidad']['tipo_accidente'].cat.add_categories(['Colisión doble', 'Otras causas', 'Choque contra obstáculo o elemento de la vía'])

colisionDoble = ['Colisión lateral', 'Colisión fronto-lateral', 'Colisión frontal']
ds['accidentalidad'].loc[ds['accidentalidad']['tipo_accidente'].isin(colisionDoble), 'tipo_accidente'] = 'Colisión doble'

otrasCausas = ['Otro', 'Despeñamiento', 'Atropello a animal', 'Solo salida de la vía']
ds['accidentalidad'].loc[ds['accidentalidad']['tipo_accidente'].isin(otrasCausas), 'tipo_accidente'] = 'Otras causas'

obstaculoFijo = ['Choque contra obstáculo fijo']
ds['accidentalidad'].loc[ds['accidentalidad']['tipo_accidente'].isin(otrasCausas), 'tipo_accidente'] = 'Choque contra obstáculo o elemento de la vía'


In [52]:
get_info('accidentalidad')

num_expediente              0
fecha                       0
hora                        0
localizacion                0
numero                      0
cod_distrito                0
distrito                    0
tipo_accidente              0
estado_meteorológico    20120
tipo_vehiculo             816
tipo_persona                0
rango_edad                  0
sexo                        0
cod_lesividad               0
lesividad                   0
coordenada_x_utm            0
coordenada_y_utm            0
positiva_alcohol          718
positiva_droga              0
cod_accidente               0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 195610 entries, 0 to 31157
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   num_expediente        195610 non-null  object  
 1   fecha                 195610 non-null  object  
 2   hora                  195610 non-null  object  
 3   localizacion

In [53]:
ds['accidentalidad']['cod_distrito'] = ds['accidentalidad']['cod_distrito'].astype('int64')
ds['accidentalidad']['cod_lesividad'] = ds['accidentalidad']['cod_lesividad'].astype('int64')
ds['accidentalidad']['tipo_accidente'] = ds['accidentalidad']['tipo_accidente'].astype('category')
ds['accidentalidad']['lesividad'] = ds['accidentalidad']['lesividad'].astype('category')
ds['accidentalidad']['sexo'] = ds['accidentalidad']['sexo'].astype('category')
ds['accidentalidad']['tipo_persona'] = ds['accidentalidad']['tipo_persona'].astype('category')
ds['accidentalidad']['distrito'] = ds['accidentalidad']['distrito'].astype('category')
ds['accidentalidad']['rango_edad'] = ds['accidentalidad']['rango_edad'].astype('category')
ds['accidentalidad']['positiva_droga'] = ds['accidentalidad']['positiva_droga'].astype('category')

ds['accidentalidad']['coordenada_x_utm'] = ds['accidentalidad']['coordenada_x_utm'].astype('str').str.replace(',', '.')
ds['accidentalidad']['coordenada_y_utm'] = ds['accidentalidad']['coordenada_y_utm'].astype('str').str.replace(',', '.')

ds['accidentalidad']['coordenada_x_utm'] = ds['accidentalidad']['coordenada_x_utm'].astype('float64')
ds['accidentalidad']['coordenada_y_utm'] = ds['accidentalidad']['coordenada_y_utm'].astype('float64')

in_proj = Proj(init='epsg:25830')  # EPSG: 25830 = 30N
out_proj = Proj(init='epsg:4326')  # sistema WGS84 delatlong

ds['accidentalidad']['longitud'], ds['accidentalidad']['latitud'] = transform(in_proj, out_proj, ds['accidentalidad']['coordenada_x_utm'].values, ds['accidentalidad']['coordenada_y_utm'].values)
ds['accidentalidad'] = ds['accidentalidad'][ds['accidentalidad']['latitud'] != 0]

ds['accidentalidad']['hora'] = ds['accidentalidad']['hora'].str.split(':')
ds['accidentalidad']['hora'] = ds['accidentalidad']['hora'].apply(lambda x: [i.zfill(2) for i in x])
ds['accidentalidad']['hora'] = ds['accidentalidad']['hora'].apply(lambda x: ':'.join(x))

ds['accidentalidad']['datetime'] = pd.to_datetime(ds['accidentalidad']['fecha'] + ' ' + ds['accidentalidad']['hora'], format='%d/%m/%Y %H:%M:%S')

ds['accidentalidad']['hora'] = ds['accidentalidad']['datetime'].dt.time
ds['accidentalidad']['fecha'] = ds['accidentalidad']['datetime'].dt.date

ds['accidentalidad']['localizacion'] = ds['accidentalidad']['localizacion'].fillna('') + ' ' + ds['accidentalidad']['numero'].fillna('')
ds['accidentalidad'].drop('numero', axis=1, inplace=True)

c:\Users\paula\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\paula\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\paula\AppData\Local\Temp\ipykernel_19560\2046809317.py:20: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/got

In [54]:
ds['accidentalidad']['positiva_alcohol'] = ds['accidentalidad']['positiva_alcohol'].str.replace('S', '1')
ds['accidentalidad']['positiva_alcohol'] = ds['accidentalidad']['positiva_alcohol'].str.replace('N', '0')
ds['accidentalidad']['positiva_alcohol'] = ds['accidentalidad']['positiva_alcohol'].astype('category')

C:\Users\paula\AppData\Local\Temp\ipykernel_19560\2377342245.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds['accidentalidad']['positiva_alcohol'] = ds['accidentalidad']['positiva_alcohol'].str.replace('S', '1')
C:\Users\paula\AppData\Local\Temp\ipykernel_19560\2377342245.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds['accidentalidad']['positiva_alcohol'] = ds['accidentalidad']['positiva_alcohol'].str.replace('N', '0')
C:\Users\paula\AppData\Local\Temp\ipykernel_19560\2377342245.py:3: Setting

In [55]:
get_info('accidentalidad')

num_expediente              0
fecha                       0
hora                        0
localizacion                0
cod_distrito                0
distrito                    0
tipo_accidente              0
estado_meteorológico    20120
tipo_vehiculo             816
tipo_persona                0
rango_edad                  0
sexo                        0
cod_lesividad               0
lesividad                   0
coordenada_x_utm            0
coordenada_y_utm            0
positiva_alcohol          718
positiva_droga              0
cod_accidente               0
longitud                    0
latitud                     0
datetime                    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 195608 entries, 0 to 31157
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   num_expediente        195608 non-null  object        
 1   fecha                 195608 non-null  o

Por último, donde queda dilema es en positivo alcohol, tipo de vehículo y estado meteorológico. En esta situación, lo más apropiado es ver si, relacionando estos atributos con algún otro, es más probable que los atributos valgan uno u otro valor.

- Estado meteorológico. Hay 7 valores posibles: despejado, lluvia débil, lluvia intensa, granizando, nevando, nublado, se desconoce. Aquí, por lo tanto, hay 3 vías de actuación:
    - Rellenar con "se desconoce", i.e.: ser fieles a lo que se sabe, reducir la proporción de datos artificiales (hay un 11% de datos faltantes), solución sencilla.
    - Rellenar con el valor más frecuente: despejado (representa el 75% de los datos), i.e.: solución con datos artificiales más sencilla.
    - Rellenar con valores aleatorios según la proporción en la que aparecen los datos, i.e.: el 75% de los datos faltantes se rellenan arbitrariamente con "Despejado".
    - Rellenar con el valor que ya hubiera ese día en esa franja horaria.

    Lo que mejor preserva los datos es, rellenar con "se desconoce", pues la variable existe previamente.

In [56]:
ds['accidentalidad'].set_index('datetime', inplace=True)

# Create a new column for the 12-hour intervals
ds['accidentalidad']['time_group'] = ds['accidentalidad'].index.to_series().dt.floor('12h')

# Group by the new column and forward fill within each group for 'estado_meteorológico' column
ds['accidentalidad']['estado_meteorológico'] = ds['accidentalidad'].groupby('time_group')['estado_meteorológico'].ffill()

# Fill any remaining NaNs with the next valid observation
ds['accidentalidad']['estado_meteorológico'] = ds['accidentalidad'].groupby('time_group')['estado_meteorológico'].bfill()

# Drop the 'time_group' column
ds['accidentalidad'].drop(columns='time_group', inplace=True)

# Infer the best data types for each column
ds['accidentalidad'] = ds['accidentalidad'].infer_objects()

# Reset the index
ds['accidentalidad'].reset_index(inplace=True)

C:\Users\paula\AppData\Local\Temp\ipykernel_19560\1423211138.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds['accidentalidad']['time_group'] = ds['accidentalidad'].index.to_series().dt.floor('12h')
C:\Users\paula\AppData\Local\Temp\ipykernel_19560\1423211138.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds['accidentalidad']['estado_meteorológico'] = ds['accidentalidad'].groupby('time_group')['estado_meteorológico'].ffill()
C:\Users\paula\AppData\Local\Temp\ipykernel_19560\1423211138.py:10: Sett

In [57]:
get_info('accidentalidad')

datetime                  0
num_expediente            0
fecha                     0
hora                      0
localizacion              0
cod_distrito              0
distrito                  0
tipo_accidente            0
estado_meteorológico      3
tipo_vehiculo           816
tipo_persona              0
rango_edad                0
sexo                      0
cod_lesividad             0
lesividad                 0
coordenada_x_utm          0
coordenada_y_utm          0
positiva_alcohol        718
positiva_droga            0
cod_accidente             0
longitud                  0
latitud                   0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195608 entries, 0 to 195607
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   datetime              195608 non-null  datetime64[ns]
 1   num_expediente        195608 non-null  object        
 2   fecha              

Como siguen quedando 3 valores NaN, los rellenamos manualmente. En caso de que fueran una cantidad significativa, repetiríamos el proceso anterior, aumentando la ventana a 24h (lo cual es equivalente a lo que vamos a hacer manualmente).

In [58]:
nan_dates = ds['accidentalidad'][ds['accidentalidad']['estado_meteorológico'].isna()]['fecha']
mask = ds['accidentalidad']['fecha'].isin(nan_dates)
estados = ds['accidentalidad'][mask].sort_values(by='datetime')

In [59]:
# fillna of ds['accidentalidad']['estado_meteorológico'] where num_expediente is 2020S006036 with Despejado
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2020S006036', 'estado_meteorológico'] = 'Despejado'
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2020S006030', 'estado_meteorológico'] = 'Despejado'

Convert 'hora' to datetime and extract the hour
ds['accidentalidad']['hora'] = pd.to_datetime(ds['accidentalidad']['hora'], format='%H:%M:%S')
ds['accidentalidad']['HOUR'] = ds['accidentalidad']['hora'].dt.hour

Categorize hours into different time periods
bins = [-1, 6, 12, 18, 24]
labels = ['Noche', 'Mañana', 'Media-Tarde', 'Tarde-Noche']
ds['accidentalidad']['TIME_OF_DAY'] = pd.cut(ds['accidentalidad']['HOUR'], bins=bins, labels=labels, include_lowest=True)

In [60]:
ds['accidentalidad']['tipo_vehiculo'].unique()

array(['Motocicleta > 125cc', 'Turismo', 'Furgoneta', 'Autobús',
       'Ciclomotor', 'Motocicleta hasta 125cc', 'Todo terreno',
       'Bicicleta', 'Camión rígido', 'Maquinaria de obras',
       'Tractocamión', nan, 'Cuadriciclo no ligero',
       'Vehículo articulado', 'Autobús articulado',
       'Otros vehículos con motor', 'Autocaravana', 'Patinete', 'Ciclo',
       'Cuadriciclo ligero', 'VMU eléctrico', 'Semiremolque',
       'Microbús <= 17 plazas', 'Sin especificar', 'Autobus EMT',
       'Remolque', 'Tranvía', 'Caravana', 'Camión de bomberos',
       'Otros vehículos sin motor', 'Bicicleta EPAC (pedaleo asistido)',
       'Moto de tres ruedas > 125cc', 'Tren/metro', 'Ambulancia SAMUR',
       'Moto de tres ruedas hasta 125cc',
       'Ciclomotor de dos ruedas L1e-B', 'Maquinaria agrícola',
       'Autobús articulado EMT', 'Ciclomotor de tres ruedas',
       'Ciclo de motor L1e-A', 'Patinete no eléctrico'], dtype=object)

- Tipo de vehículo. Hay 34 valores posibles: Ambulancia SAMUR, autobús EMT, autobús, autobús articulado, autobús articulado EMT, autocaravana, bicicleta, bicicleta EPAC (pedaleo asistido), camión de bomberos, camión rígido, ciclo, ciclomotor, ciclomotor de dos ruedas L1e-B, cuadriciclo ligero, cuadriciclo no ligero, furgoneta, maquinaria de obras, microbús <= 17 plazas, moto de tres ruedas > 125cc, moto de tres ruedas hasta 125cc, motocicleta > 125cc, motocicleta hasta 125cc, otros vehículos con motor, otros vehículos sin motor, patinete no eléctrico, remolque, semirremolque, sin especificar, todo terreno, tractocamión, tren/metro, turismo (68%), VMU eléctrico, vehículo articulado. En esta variable hay 0.6% de valores faltantes, lo cual no es significativo, i.e.: la sustitución que elijamos tendrá menos repercusión en el estudio final. Aquí, por lo tanto, hay 2 vías de actuación:
    - Rellenar con "sin especificar", i.e.: solución sencilla y descriptiva pero que puede dar lugar a interpretaciones erróneas, pues puede haber sido otro tipo de vehículo que no se había registrado.
    - Rellenar con el valor más probable según otro atributo (por ejemplo, código de lesividad).
    
    La solución más apropiada es rellenar con el valor más probable según código de lesividad, por lo que vamos a ver primero cómo se relacionan ambos atributos y después rellenaremos los valores faltantes con el valor más probable.

In [61]:
import scipy.stats as stats
contingency_table = pd.crosstab(ds['accidentalidad']['tipo_vehiculo'], ds['accidentalidad']['cod_lesividad'])
chi2, p, _, _ = stats.chi2_contingency(contingency_table)
if (p < 0.05):
    print('Tipo de vehículo y lesividad están relacionados')

Tipo de vehículo y lesividad están relacionados


Por lo tanto, es coherente rellenar el tipo de vehiculo según el grado de lesividad

In [62]:
ds['accidentalidad']['tipo_vehiculo'] = ds['accidentalidad']['tipo_vehiculo'].fillna(ds['accidentalidad'].groupby('cod_lesividad')['tipo_vehiculo'].transform(lambda x:x.mode().iat[0]))

Para rellenar los datos de positivo en alcohol, usaremos lo más habitual según grupo de edad, sexo y lesividad de la persona

In [63]:
ds['accidentalidad']['positiva_alcohol'] = ds['accidentalidad'].groupby(['rango_edad','sexo','lesividad'])['positiva_alcohol'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty                                                                                                        else "Empty"))

C:\Users\paula\AppData\Local\Temp\ipykernel_19560\1668006814.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  ds['accidentalidad']['positiva_alcohol'] = ds['accidentalidad'].groupby(['rango_edad','sexo','lesividad'])['positiva_alcohol'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty                                                                                                        else "Empty"))


In [64]:
ds['accidentalidad'] = ds['accidentalidad'][ds['accidentalidad'].distrito != '13.0']

In [65]:
ds['accidentalidad'] = ds['accidentalidad'].loc[ds['accidentalidad']['tipo_accidente'] != 13.0]
ds['accidentalidad'].drop(columns=['coordenada_x_utm', 'coordenada_y_utm'], inplace=True)

In [66]:
get_info('accidentalidad')

datetime                0
num_expediente          0
fecha                   0
hora                    0
localizacion            0
cod_distrito            0
distrito                0
tipo_accidente          0
estado_meteorológico    0
tipo_vehiculo           0
tipo_persona            0
rango_edad              0
sexo                    0
cod_lesividad           0
lesividad               0
positiva_alcohol        0
positiva_droga          0
cod_accidente           0
longitud                0
latitud                 0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 195608 entries, 0 to 195607
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   datetime              195608 non-null  datetime64[ns]
 1   num_expediente        195608 non-null  object        
 2   fecha                 195608 non-null  object        
 3   hora                  195608 non-null  object        
 4   lo

In [67]:
ds['accidentalidad']['tipo_vehiculo'] = ds['accidentalidad']['tipo_vehiculo'].astype('category')
ds['accidentalidad']['positiva_alcohol'] = ds['accidentalidad']['positiva_alcohol'].astype('category')
ds['accidentalidad']['estado_meteorológico'] = ds['accidentalidad']['estado_meteorológico'].astype('category')

In [68]:
get_info('accidentalidad')

datetime                0
num_expediente          0
fecha                   0
hora                    0
localizacion            0
cod_distrito            0
distrito                0
tipo_accidente          0
estado_meteorológico    0
tipo_vehiculo           0
tipo_persona            0
rango_edad              0
sexo                    0
cod_lesividad           0
lesividad               0
positiva_alcohol        0
positiva_droga          0
cod_accidente           0
longitud                0
latitud                 0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 195608 entries, 0 to 195607
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   datetime              195608 non-null  datetime64[ns]
 1   num_expediente        195608 non-null  object        
 2   fecha                 195608 non-null  object        
 3   hora                  195608 non-null  object        
 4   lo

Con esto, queda limpio el dataset de accidentalidad

In [69]:
ds['accidentalidad'].to_csv('limpios/accidentalidad.csv', index=False)

#### Aforo del tráfico

In [70]:
get_info('estaciones')
ds['estaciones']

fdia     0
fest     0
fsen     0
hor1     0
hor2     0
hor3     0
hor4     0
hor5     0
hor6     0
hor7     0
hor8     0
hor9     0
hor10    0
hor11    0
hor12    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 251104 entries, 0 to 7079
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   fdia    251104 non-null  object 
 1   fest    251104 non-null  object 
 2   fsen    251104 non-null  object 
 3   hor1    251104 non-null  float64
 4   hor2    251104 non-null  float64
 5   hor3    251104 non-null  float64
 6   hor4    251104 non-null  float64
 7   hor5    251104 non-null  float64
 8   hor6    251104 non-null  float64
 9   hor7    251104 non-null  float64
 10  hor8    251104 non-null  float64
 11  hor9    251104 non-null  float64
 12  hor10   251104 non-null  float64
 13  hor11   251104 non-null  float64
 14  hor12   251104 non-null  float64
dtypes: float64(12), object(3)
memory usage: 30.7+ MB
None


,fdia,fest,fsen,hor1,hor2,hor3,hor4,hor5,hor6,hor7,hor8,hor9,hor10,hor11,hor12
0,01/04/2021,ES01,1-,38.0,32.0,25.0,17.0,46.0,165.0,176.0,218.0,331.0,467.0,662.0,811.0
1,01/04/2021,ES01,1=,944.0,856.0,577.0,720.0,886.0,952.0,1067.0,1100.0,941.0,1029.0,724.0,86.0
2,01/04/2021,ES01,2-,62.0,27.0,24.0,37.0,31.0,130.0,126.0,232.0,330.0,606.0,771.0,978.0
3,01/04/2021,ES01,2=,1084.0,912.0,608.0,824.0,1019.0,1209.0,1089.0,949.0,724.0,685.0,597.0,147.0
4,01/04/2021,ES02,1-,221.0,80.0,55.0,44.0,53.0,48.0,80.0,138.0,184.0,260.0,335.0,430.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7075,30/09/2023,ES59,2=,174.0,179.0,141.0,169.0,134.0,176.0,214.0,148.0,179.0,93.0,181.0,68.0
7076,30/09/2023,ES60,1-,121.0,96.0,50.0,58.0,70.0,94.0,119.0,195.0,325.0,432.0,438.0,507.0
7077,30/09/2023,ES60,1=,448.0,352.0,250.0,239.0,292.0,315.0,326.0,365.0,276.0,198.0,163.0,190.0
7078,30/09/2023,ES60,2-,127.0,101.0,62.0,56.0,36.0,64.0,87.0,146.0,239.0,367.0,380.0,463.0


Vamos a realizar transformaicones en el formato del dataset para facilitar su interpretación. Para ello, primero de todo, cambiamos los datos a los tipos correspondientes.

In [71]:
ds['estaciones']['fdia'] = pd.to_datetime(ds['estaciones']['fdia'], format='mixed', dayfirst=True)

for i in range(1, 13):
    ds['estaciones'][f'hor{i}'] = ds['estaciones'][f'hor{i}'].astype('int64')


In [72]:
get_info('estaciones')

fdia     0
fest     0
fsen     0
hor1     0
hor2     0
hor3     0
hor4     0
hor5     0
hor6     0
hor7     0
hor8     0
hor9     0
hor10    0
hor11    0
hor12    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 251104 entries, 0 to 7079
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   fdia    251104 non-null  datetime64[ns]
 1   fest    251104 non-null  object        
 2   fsen    251104 non-null  object        
 3   hor1    251104 non-null  int64         
 4   hor2    251104 non-null  int64         
 5   hor3    251104 non-null  int64         
 6   hor4    251104 non-null  int64         
 7   hor5    251104 non-null  int64         
 8   hor6    251104 non-null  int64         
 9   hor7    251104 non-null  int64         
 10  hor8    251104 non-null  int64         
 11  hor9    251104 non-null  int64         
 12  hor10   251104 non-null  int64         
 13  hor11   251104 non-null  int64        

Este dataset no tiene datos faltantes. Apreciamos que FSEN, aparte de indicar el sentido del tráfico, también indica si son las primeras 12h del día (1- y 2-), o las últimas (1=, 2=). Hacemos 12 nuevos atributos para suplir esto en un formato 24h que es más cómodo.

In [73]:
df = ds['estaciones']

df[['direccion', 'ampm']] = df['fsen'].str.extract(r'(\d)([=-])')
df.drop(['fsen'], axis=1, inplace=True)

index = ['fdia', 'fest', 'direccion', 'ampm', 'hor1', 'hor2', 'hor3', 'hor4', 'hor5', 'hor6', 'hor7', 'hor8', 'hor9', 'hor10', 'hor11', 'hor12', 'hor13', 'hor14', 'hor15', 'hor16', 'hor17', 'hor18', 'hor19', 'hor20', 'hor21', 'hor22', 'hor23', 'hor24']

df.reindex(columns=index)

,fdia,fest,direccion,ampm,hor1,hor2,hor3,hor4,hor5,hor6,...,hor15,hor16,hor17,hor18,hor19,hor20,hor21,hor22,hor23,hor24
0,2021-04-01,ES01,1,-,38,32,25,17,46,165,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-04-01,ES01,1,=,944,856,577,720,886,952,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-04-01,ES01,2,-,62,27,24,37,31,130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-04-01,ES01,2,=,1084,912,608,824,1019,1209,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-04-01,ES02,1,-,221,80,55,44,53,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7075,2023-09-30,ES59,2,=,174,179,141,169,134,176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7076,2023-09-30,ES60,1,-,121,96,50,58,70,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7077,2023-09-30,ES60,1,=,448,352,250,239,292,315,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7078,2023-09-30,ES60,2,-,127,101,62,56,36,64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Ahora procedemos a la copia de los valores de las columnas en las horas de la tarde en aquellas filas en las que sea necesario

In [74]:
am = df[df['ampm'] != '='].copy()
pm = df[df['ampm'] == '='].copy()

# Shift the values in pm from 'hor1'-'hor12' to 'hor13'-'hor24'
for i in range(1, 13):
    pm.loc[:, f'hor{i+12}'] = pm.loc[:, f'hor{i}'].astype(float)
    pm.loc[:, f'hor{i}'] = np.nan

df = pd.concat([am, pm])
df = df.drop(columns=['ampm'])

# Combine rows that share the same fdia, fest, direccion
df = df.groupby(['fdia', 'fest', 'direccion'], as_index=False).first()
ds['estaciones'] = df
del df

ds['estaciones']


C:\Users\paula\AppData\Local\Temp\ipykernel_19560\2868808003.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'nan' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  pm.loc[:, f'hor{i}'] = np.nan
C:\Users\paula\AppData\Local\Temp\ipykernel_19560\2868808003.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'nan' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  pm.loc[:, f'hor{i}'] = np.nan
C:\Users\paula\AppData\Local\Temp\ipykernel_19560\2868808003.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'nan' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  pm.loc[:, f'hor{i}'] = np.nan
C:\Users\paula\AppData\Local\Temp\ipykernel_19560\2868808003.py:7: Future

,fdia,fest,direccion,hor1,hor2,hor3,hor4,hor5,hor6,hor7,...,hor15,hor16,hor17,hor18,hor19,hor20,hor21,hor22,hor23,hor24
0,2021-01-01,ES01,1,643.0,87.0,31.0,28.0,24.0,118.0,98.0,...,332.0,470.0,555.0,969.0,1633.0,1510.0,948.0,570.0,472.0,155.0
1,2021-01-01,ES01,2,606.0,123.0,48.0,37.0,28.0,89.0,106.0,...,340.0,399.0,656.0,1242.0,1360.0,1026.0,660.0,381.0,343.0,190.0
2,2021-01-01,ES02,1,161.0,490.0,193.0,81.0,46.0,56.0,77.0,...,370.0,315.0,363.0,399.0,546.0,0.0,643.0,459.0,372.0,301.0
3,2021-01-01,ES02,2,264.0,134.0,71.0,62.0,61.0,165.0,555.0,...,919.0,888.0,917.0,1063.0,1036.0,1131.0,1131.0,1091.0,853.0,589.0
4,2021-01-01,ES03,1,391.0,1076.0,184.0,77.0,77.0,57.0,110.0,...,958.0,473.0,385.0,562.0,704.0,0.0,665.0,694.0,429.0,273.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125547,2023-11-30,ES58,2,20.0,18.0,8.0,11.0,35.0,60.0,199.0,...,225.0,272.0,420.0,324.0,286.0,267.0,185.0,124.0,73.0,43.0
125548,2023-11-30,ES59,1,33.0,29.0,23.0,23.0,50.0,103.0,162.0,...,283.0,319.0,342.0,330.0,328.0,271.0,185.0,158.0,140.0,63.0
125549,2023-11-30,ES59,2,17.0,26.0,18.0,25.0,48.0,150.0,196.0,...,190.0,199.0,232.0,228.0,226.0,179.0,129.0,87.0,63.0,37.0
125550,2023-11-30,ES60,1,35.0,24.0,22.0,28.0,35.0,175.0,606.0,...,379.0,454.0,579.0,496.0,461.0,360.0,208.0,138.0,116.0,90.0


In [75]:
get_info('estaciones')

fdia         0
fest         0
direccion    0
hor1         0
hor2         0
hor3         0
hor4         0
hor5         0
hor6         0
hor7         0
hor8         0
hor9         0
hor10        0
hor11        0
hor12        0
hor13        0
hor14        0
hor15        0
hor16        0
hor17        0
hor18        0
hor19        0
hor20        0
hor21        0
hor22        0
hor23        0
hor24        0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125552 entries, 0 to 125551
Data columns (total 27 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   fdia       125552 non-null  datetime64[ns]
 1   fest       125552 non-null  object        
 2   direccion  125552 non-null  object        
 3   hor1       125552 non-null  float64       
 4   hor2       125552 non-null  float64       
 5   hor3       125552 non-null  float64       
 6   hor4       125552 non-null  float64       
 7   hor5       125552 non-null  float6

In [76]:
ds['estaciones']['fest'] = ds['estaciones']['fest'].str.extract(r'(\d+)').astype('int64')
ds['estaciones']['direccion'] = ds['estaciones']['direccion'].astype('int64')
for i in range(1, 25):
    ds['estaciones'][f'hor{i}'] = ds['estaciones'][f'hor{i}'].astype('int64')
get_info('estaciones')

fdia         0
fest         0
direccion    0
hor1         0
hor2         0
hor3         0
hor4         0
hor5         0
hor6         0
hor7         0
hor8         0
hor9         0
hor10        0
hor11        0
hor12        0
hor13        0
hor14        0
hor15        0
hor16        0
hor17        0
hor18        0
hor19        0
hor20        0
hor21        0
hor22        0
hor23        0
hor24        0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125552 entries, 0 to 125551
Data columns (total 27 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   fdia       125552 non-null  datetime64[ns]
 1   fest       125552 non-null  int64         
 2   direccion  125552 non-null  int64         
 3   hor1       125552 non-null  int64         
 4   hor2       125552 non-null  int64         
 5   hor3       125552 non-null  int64         
 6   hor4       125552 non-null  int64         
 7   hor5       125552 non-null  int64 

In [77]:
ds['estaciones']['fest'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60], dtype=int64)

In [78]:
get_info('nombres_estaciones')
ds['nombres_estaciones']

estación     0
nombre       0
latitud      0
longitud     0
sentido     10
orient.     10
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 119 entries, 0 to 119
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   estación  119 non-null    int64  
 1   nombre    119 non-null    object 
 2   latitud   119 non-null    object 
 3   longitud  119 non-null    object 
 4   sentido   109 non-null    float64
 5   orient.   109 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.5+ KB
None


,estación,nombre,latitud,longitud,sentido,orient.
0,1,Paseo de la Castellana,"40,4319272588958","-3,68910874956933",1.0,S-N
1,1,Paseo de la Castellana,"40,4319272588958","-3,68910874956933",2.0,N-S
2,2,Calle Princesa,"40,4260679064732","-3,71270896289063",1.0,N-S
3,2,Calle Princesa,"40,4260679064732","-3,71270896289063",2.0,S-N
4,3,Calle Doctor Esquerdo,"40,4175763959444","-3,66926245767642",1.0,S-N
...,...,...,...,...,...,...
115,58,Avenida de Logroño,"40,4618490894087","-3,5910937723728",2.0,O-E
116,59,Calle San Cipriano,"40,4044120727823","-3,60588835825753",1.0,O-E
117,59,Calle San Cipriano,"40,4044120727823","-3,60588835825753",2.0,E-O
118,60,Calle Camino de Vinateros,"40,4109315826646","-3,65683389839975",1.0,E-O


In [79]:
ds['nombres_estaciones']['sentido'] = ds['nombres_estaciones']['sentido'].fillna(0)
ds['nombres_estaciones']['sentido'] = ds['nombres_estaciones']['sentido'].astype('int64')

In [80]:
ds['nombres_estaciones']['latitud'] = ds['nombres_estaciones']['latitud'].str.replace(',', '.').str.replace('.', '').astype('float64')
ds['nombres_estaciones']['longitud'] = ds['nombres_estaciones']['longitud'].str.replace(',', '.').str.replace('.', '').astype('float64')

In [81]:
ds['nombres_estaciones'][ds['nombres_estaciones'].isna().any(axis=1)]['estación'].unique()

array([ 9, 10, 12, 17, 19, 25, 34, 35, 44, 45], dtype=int64)

Ahora que ya está todo, podemos fusionar ambos datasets, para poder tener un df con todos los datos del tráfico, incluidas las localizaciones y nombres de las estaciones.

In [82]:
ds['nombres_estaciones'].to_csv('limpios/estacionesNombres.csv', index=False)

In [83]:
df = ds['estaciones'].rename(columns={'fest': 'estación', 'direccion': 'sentido'})
ds['nombres_estaciones'] = ds['nombres_estaciones'].merge(df, on=['estación', 'sentido'])
del df

In [84]:
df_melted = ds['nombres_estaciones'].melt(id_vars=['estación', 'nombre', 'latitud', 'longitud', 'sentido', 'orient.', 'fdia'], 
                    var_name='hour', 
                    value_name='aforo')

df_melted['hour'] = df_melted['hour'].str.extract('(\d+)').astype(int)


<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\paula\AppData\Local\Temp\ipykernel_19560\238365233.py:5: SyntaxWarning: invalid escape sequence '\d'
  df_melted['hour'] = df_melted['hour'].str.extract('(\d+)').astype(int)


In [85]:
df_melted['hour_str'] = df_melted['hour'].apply(lambda x: f'{x:02d}:00:00' if x != 24 else '00:00:00')
del df_melted['hour']

In [86]:
df_melted.head()

,estación,nombre,latitud,longitud,sentido,orient.,fdia,aforo,hour_str
0,1,Paseo de la Castellana,4.043193e+14,-3.689109e+14,1,S-N,2021-01-01,643,01:00:00
1,1,Paseo de la Castellana,4.043193e+14,-3.689109e+14,1,S-N,2021-01-02,38,01:00:00
2,1,Paseo de la Castellana,4.043193e+14,-3.689109e+14,1,S-N,2021-01-03,61,01:00:00
3,1,Paseo de la Castellana,4.043193e+14,-3.689109e+14,1,S-N,2021-01-04,31,01:00:00
4,1,Paseo de la Castellana,4.043193e+14,-3.689109e+14,1,S-N,2021-01-05,24,01:00:00


In [87]:
df_melted.tail()

,estación,nombre,latitud,longitud,sentido,orient.,fdia,aforo,hour_str
2783419,60,Calle Camino de Vinateros,4.041093e+14,-3.656834e+14,2,O-E,2023-11-26,73,00:00:00
2783420,60,Calle Camino de Vinateros,4.041093e+14,-3.656834e+14,2,O-E,2023-11-27,104,00:00:00
2783421,60,Calle Camino de Vinateros,4.041093e+14,-3.656834e+14,2,O-E,2023-11-28,105,00:00:00
2783422,60,Calle Camino de Vinateros,4.041093e+14,-3.656834e+14,2,O-E,2023-11-29,103,00:00:00
2783423,60,Calle Camino de Vinateros,4.041093e+14,-3.656834e+14,2,O-E,2023-11-30,135,00:00:00


In [88]:
df_melted['datetime'] = df_melted['fdia'].astype(str) + ' ' + df_melted['hour_str'].astype(str)

In [89]:
df = df_melted[['datetime', 'nombre', 'sentido', 'longitud', 'latitud', 'aforo', 'orient.', 'hour_str']].rename(columns={'estación': 'estacion'})
del df_melted
df.to_csv('limpios/estacionesCruzadas.csv', index=False)
ds['estaciones'].to_csv('limpios/estaciones.csv', index=False)

#### Direcciones

In [90]:
get_info('direcciones')

cod_via                    0
via_clase                  0
via_par                 4466
via_nombre                 0
via_nombre_acentos         0
clase_app                  0
numero                     0
calificador           178799
tipo_ndp                   0
cod_ndp                    0
distrito                   0
barrio                     2
cod_postal                 0
utmx_ed                    0
utmy_ed                    0
utmx_etrs                  0
utmy_etrs                  0
latitud                    0
longitud                   0
angulo_rotulacion          0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 211687 entries, 0 to 211966
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   cod_via             211687 non-null  int64  
 1   via_clase           211687 non-null  object 
 2   via_par             207221 non-null  object 
 3   via_nombre          211687 non-null  object 
 

Como podemos observar, faltan datos de las columnas via_par (partícula de la denominación), con 4466; calificador (del número de la policía), con 178799; barrio con 2.

In [91]:
ds['direcciones']

,cod_via,via_clase,via_par,via_nombre,via_nombre_acentos,clase_app,numero,calificador,tipo_ndp,cod_ndp,distrito,barrio,cod_postal,utmx_ed,utmy_ed,utmx_etrs,utmy_etrs,latitud,longitud,angulo_rotulacion
0,31001337,AUTOVÍA,NaN,A-1,A-1,KILÓMETRO,10000,EN,PARCELA,31031089,8,6.0,28050,44305633,44825034,44294695,448229592,40°29'21.84'',3°40'23.56'',78.87
1,31001337,AUTOVÍA,NaN,A-1,A-1,KILÓMETRO,10000,SA,PARCELA,31031088,16,6.0,28050,44312246,448249077,44301308,448228329,40°29'21.45'',3°40'20.75'',77.25
2,31001337,AUTOVÍA,NaN,A-1,A-1,KILÓMETRO,11000,EN,PARCELA,31031091,8,6.0,28050,44367522,448330933,44356584,448310185,40°29'48.13'',3°39'57.53'',41.35
3,31001337,AUTOVÍA,NaN,A-1,A-1,KILÓMETRO,11000,SA,PARCELA,31031090,16,6.0,28050,44373755,448325151,44362817,448304402,40°29'46.27'',3°39'54.87'',47.56
4,31001337,AUTOVÍA,NaN,A-1,A-1,KILÓMETRO,12000,EN,PARCELA,31031093,8,6.0,28050,44425207,448408185,4441427,448387436,40°30'13.33'',3°39'33.27'',67.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211962,725,PASEO,DEL,ZURRON,ZURRÓN,NUMERO,49,NaN,FRENTE FACHADA,20045135,21,4.0,28042,45051964,448025888,45041023,448005128,40°28'10.77'',3°35'5.92'',358.91
211963,725,PASEO,DEL,ZURRON,ZURRÓN,NUMERO,51,NaN,FRENTE FACHADA,20045138,21,4.0,28042,4505808,44802596,45047139,448005199,40°28'10.81'',3°35'3.32'',359.64
211964,725,PASEO,DEL,ZURRON,ZURRÓN,NUMERO,53,NaN,PORTAL,20045141,21,4.0,28042,45061025,448027575,45050084,448006814,40°28'11.34'',3°35'2.07'',87.37
211965,725,PASEO,DEL,ZURRON,ZURRÓN,NUMERO,55,NaN,FRENTE FACHADA,20045144,21,4.0,28042,450635,44802708,45052558,448006319,40°28'11.19'',3°35'1.02'',355.97


La partícula de denominación (VIA_PAR), hay 7 valores distintos de entre la lista posible de 20 (DE, EL, A, DEL, LA, AL, DE LA, LAS, A LA, DE LAS, LO, A LAS, DE LO, LOS, A LO, DE LO, DE LOS, POR EL, A LOS, POR LA), de los cuales el primero es A LA y el último es DEL. Por lo tanto, parece que no se ha rellenado con Null o espacio vacío aquellas direcciones en las que no hay partícula:

In [92]:
ds['direcciones']['via_par'] = ds['direcciones']['via_par'].fillna(' ')

Ahora, juntamos todas las columnas asociadas a la direccion en un único atributo: direccion

In [93]:
ds['direcciones']['direccion'] = ds['direcciones']['via_clase'] + " " + ds['direcciones']['via_par'] + " " + ds['direcciones']['via_nombre'] + " " + ds['direcciones']['clase_app'] + " " + ds['direcciones']['numero'].astype(str)
ds['direcciones'] = ds['direcciones'].drop(columns=['via_par', 'via_nombre', 'clase_app', 'numero', 'via_nombre_acentos'])

Para Calificador, como no sabemos realmente cuál es el criterio y la mayoría de datos son faltantes (322826 faltan de 371584), lo más prudente es eliminar la columna directamente

In [94]:
ds['direcciones'] = ds['direcciones'].drop('calificador', axis=1)
ds['direcciones'] = ds['direcciones'].drop('angulo_rotulacion', axis=1)
ds['direcciones'] = ds['direcciones'].filter(regex='^(?!ut)')

In [95]:
def dms_to_decimal(dms):
    parts = dms.replace('°', ' ').replace("'", ' ').replace('"', ' ').split()
    degrees = float(parts[0])
    minutes = float(parts[1]) if len(parts) > 1 else 0.0
    seconds = float(parts[2]) if len(parts) > 2 else 0.0
    return degrees + minutes/60 + seconds/3600

ds['direcciones']['latitud'] = ds['direcciones']['latitud'].apply(dms_to_decimal)
ds['direcciones']['longitud'] = ds['direcciones']['longitud'].apply(dms_to_decimal)

In [96]:
ds['direcciones']['longitud'] = ds['direcciones']['longitud'] * -1

In [97]:
get_info('direcciones')

cod_via       0
via_clase     0
tipo_ndp      0
cod_ndp       0
distrito      0
barrio        2
cod_postal    0
latitud       0
longitud      0
direccion     0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 211687 entries, 0 to 211966
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   cod_via     211687 non-null  int64  
 1   via_clase   211687 non-null  object 
 2   tipo_ndp    211687 non-null  object 
 3   cod_ndp     211687 non-null  int64  
 4   distrito    211687 non-null  int64  
 5   barrio      211685 non-null  float64
 6   cod_postal  211687 non-null  int64  
 7   latitud     211687 non-null  float64
 8   longitud    211687 non-null  float64
 9   direccion   211687 non-null  object 
dtypes: float64(3), int64(4), object(3)
memory usage: 17.8+ MB
None


In [98]:
ds['direcciones'][ds['direcciones'].isna().any(axis=1)]

,cod_via,via_clase,tipo_ndp,cod_ndp,distrito,barrio,cod_postal,latitud,longitud,direccion
120148,453600,CALLE,GARAJE,31007309,43,NaN,28028,40.427339,-3.663642,CALLE DEL MAESTRO ALONSO NUMERO 26
162447,31005958,CALLE,PARCELA,31071862,0,NaN,0,40.375122,-3.585872,CALLE PROVISIONAL BERROCALES SESENTA Y NUEVE...


In [99]:
ds['direcciones'][ds['direcciones']['direccion'].str.startswith('CALLE DEL MAESTRO ALONSO')]['barrio'].unique()[0]


3.0

In [100]:
ds['direcciones'].loc[ds['direcciones']['direccion'] == 'CALLE DEL MAESTRO ALONSO NUMERO 26', 'barrio'] = 3.0

In [101]:
ds['direcciones'].loc[ds['direcciones']['direccion'] == 'CALLE PROVISIONAL BERROCALES SESENTA Y NUEVE...', 'cod_postal'] = 28031
ds['direcciones'].loc[162447, 'distrito'] = 19
ds['direcciones'][ds['direcciones']['cod_postal'] == 28031]

,cod_via,via_clase,tipo_ndp,cod_ndp,distrito,barrio,cod_postal,latitud,longitud,direccion
48,31001342,AUTOVÍA,PARCELA,31031249,19,2.0,28031,40.389839,-3.616586,AUTOVÍA A-3 KILÓMETRO 8000
49,31001342,AUTOVÍA,PARCELA,31031248,18,2.0,28031,40.389483,-3.616683,AUTOVÍA A-3 KILÓMETRO 8000
50,31001342,AUTOVÍA,PARCELA,31051825,19,2.0,28031,40.388197,-3.609600,AUTOVÍA A-3 KILÓMETRO 8600
51,31001342,AUTOVÍA,PARCELA,31031251,18,2.0,28031,40.386258,-3.606275,AUTOVÍA A-3 KILÓMETRO 9000
52,31001342,AUTOVÍA,PARCELA,31031250,18,2.0,28031,40.386086,-3.606558,AUTOVÍA A-3 KILÓMETRO 9000
...,...,...,...,...,...,...,...,...,...,...
211430,804850,CALLE,JARDÍN/PARQUE,31029134,18,2.0,28031,40.381714,-3.607336,CALLE DE ZAZUAR NUMERO 15
211431,804850,CALLE,JARDÍN/PARQUE,31029135,18,2.0,28031,40.381853,-3.607817,CALLE DE ZAZUAR NUMERO 15
211432,804850,CALLE,PORTAL,11131637,18,2.0,28031,40.381528,-3.608119,CALLE DE ZAZUAR NUMERO 17
211433,804850,CALLE,PARCELA,20042034,18,2.0,28031,40.381236,-3.607731,CALLE DE ZAZUAR NUMERO 19


In [102]:
ds['direcciones'].loc[162447, 'barrio'] = 2.0

In [103]:
get_info('direcciones')

cod_via       0
via_clase     0
tipo_ndp      0
cod_ndp       0
distrito      0
barrio        0
cod_postal    0
latitud       0
longitud      0
direccion     0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 211687 entries, 0 to 211966
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   cod_via     211687 non-null  int64  
 1   via_clase   211687 non-null  object 
 2   tipo_ndp    211687 non-null  object 
 3   cod_ndp     211687 non-null  int64  
 4   distrito    211687 non-null  int64  
 5   barrio      211687 non-null  float64
 6   cod_postal  211687 non-null  int64  
 7   latitud     211687 non-null  float64
 8   longitud    211687 non-null  float64
 9   direccion   211687 non-null  object 
dtypes: float64(3), int64(4), object(3)
memory usage: 25.8+ MB
None


In [104]:
ds['direcciones']['barrio'] = ds['direcciones']['barrio'].astype('int64')

In [105]:
ds['direcciones'].to_csv('limpios/direcciones.csv', index=False)

#### Radares

Pasamos al dataset estrella: radares.

In [106]:
get_info('radares')
ds['radares']

radar                     0
ubicacion                 0
carretara_o_vial          1
ubicación\r\ncalle_30     8
pk                        1
sentido                   3
tipo                      3
x_(wgs84)                13
y_(wgs84)                13
longitud                  0
latitud                   0
coordenadas               0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   radar                29 non-null     int64  
 1   ubicacion            29 non-null     object 
 2   carretara_o_vial     28 non-null     object 
 3   ubicación
calle_30  21 non-null     object 
 4   pk                   28 non-null     object 
 5   sentido              26 non-null     object 
 6   tipo                 26 non-null     object 
 7   x_(wgs84)            16 non-null     float64
 8   y_(wgs84)            16 non-null     float64

,radar,ubicacion,carretara_o_vial,ubicación\r\ncalle_30,pk,sentido,tipo,x_(wgs84),y_(wgs84),longitud,latitud,coordenadas
0,1,"M-30, ENTRADA DE LA A-1, SENTIDO PUENTE DE VEN...",M-30,00NC50,0+500,Calzada interior,Fijo carriles 3 y 4,442843.0000,4481180.000,-3.674338,40.479341,DGP
1,2,"M-30, ENTRADA DE LA A-1, SENTIDO PUENTE DE VEN...",M-30,00NC50,0+500,Calzada interior,Fijo carriles 3 y 4,442847.0000,4481180.000,-3.674291,40.479342,DGP
2,3,"PASO INFERIOR COSTA RICA, SENTIDO PRÍNCIPE DE ...",NaN,NaN,NaN,NaN,NaN,443295.0000,4478910.000,-3.668803,40.458923,DGP
3,4,"M-30, PTE. DE VENTAS. SENTIDO SUR A O´DONNELL ...",M-30,M-30,6+700,Tronco calzada interior,Fijo carriles 1 y 3,444002.0000,4475270.000,-3.660144,40.426180,DGP
4,5,"M-30, PTE. DE VENTAS. SENTIDO SUR A O´DONNELL ...",M-30,M-30,6+700,Tronco calzada interior,Fijo carriles 1 y 3,444009.0000,4475271.000,-3.660061,40.426190,DGP
5,6,"M-30, ENLACE O´DONNELL SENTIDO NORTE A PTE. DE...",M-30,M-30,7+800,Tronco calzada exterior,Fijo carriles 2 y 3,444069.0000,4474210.000,-3.659261,40.416636,DGP
6,7,"M-30, ENLACE O´DONNELL SENTIDO NORTE A PTE. DE...",M-30,M-30,7+800,Tronco calzada exterior,Fijo carriles 2 y 3,444066.0000,4474211.000,-3.659296,40.416644,DGP
7,8,"TÚNEL BAIPÁS, SENTIDO NORTE A PTE. DE VENTAS (...",M-30,10XC30,10+300,Calzada exterior,Fijo carril 1,NaN,NaN,-3.670048,40.397352,Calle30
8,9,"TÚNEL BAIPÁS, SENTIDO NORTE A PTE. DE VENTAS (...",M-30,10XC30,10+300,Calzada exterior,Fijo carril 3,NaN,NaN,-3.669973,40.397323,Calle30
9,10,"ENTRADA TÚNEL M-30, DESDE NUDO SUR HACIA PTE. ...",M-30,13NL25,13+250,Calzada interior,Fijo carril 3,NaN,NaN,-3.694998,40.386787,Calle30


In [107]:
ds['radares'] = ds['radares'].drop(columns=['x_(wgs84)', 'y_(wgs84)', 'ubicación\r\ncalle_30', 'coordenadas'])

In [108]:
get_info('radares')

radar               0
ubicacion           0
carretara_o_vial    1
pk                  1
sentido             3
tipo                3
longitud            0
latitud             0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   radar             29 non-null     int64  
 1   ubicacion         29 non-null     object 
 2   carretara_o_vial  28 non-null     object 
 3   pk                28 non-null     object 
 4   sentido           26 non-null     object 
 5   tipo              26 non-null     object 
 6   longitud          29 non-null     float64
 7   latitud           29 non-null     float64
dtypes: float64(2), int64(1), object(5)
memory usage: 1.9+ KB
None


In [109]:
ds['radares'][ds['radares'].isna().any(axis=1)]

,radar,ubicacion,carretara_o_vial,pk,sentido,tipo,longitud,latitud
2,3,"PASO INFERIOR COSTA RICA, SENTIDO PRÍNCIPE DE ...",NaN,NaN,NaN,NaN,-3.668803,40.458923
12,13,"PASEO DE STA. Mª DE LA CABEZA Nº 115, ENTRADA",PASEO DE SANTA MARIA DE LA CABEZA,115,NaN,NaN,-3.710177,40.390523
19,20,"A-5 PK 4,000, SALIDA",A5,4,Salida,NaN,-3.745452,40.407445
24,25,M-30 SENTIDO CTRA DEL PARDO (AVDA. DE LA ILUS...,M-30,27+300,NaN,Fijo multicarril,-3.730897,40.472790


In [110]:
ds['radares'].at[12, 'sentido'] = 'Entrada'
pasoInferior = ds['radares'].at[2, 'ubicacion']
ubicacion, sentido = pasoInferior.split(',')
sentido = sentido.split(' ', 2)[2]
ds['radares'].at[2, 'sentido'] = sentido
ds['radares'].at[2, 'ubicacion'] = ubicacion
ds['radares'].at[24, 'ubicacion']
ds['radares'].at[24, 'ubicacion'] = 'M-30 CTRA DEL PARDO (AVDA. DE LA ILUSTRACIÓN - CTRA. DEL PARDO)'
ds['radares'].at[24, 'sentido'] = 'CTRA DEL PARDO'
ds['radares'].at[19, 'tipo'] = 'Radar de tramo'
ds['radares'].at[12, 'tipo'] = 'Fijo'
ds['radares'].at[2, 'tipo'] = 'Fijo'
ds['radares'].at[2, 'carretara_o_vial'] = 'M-30'
ds['radares'].at[2, 'pk'] = '-'

In [111]:
get_info('radares')

radar               0
ubicacion           0
carretara_o_vial    0
pk                  0
sentido             0
tipo                0
longitud            0
latitud             0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   radar             29 non-null     int64  
 1   ubicacion         29 non-null     object 
 2   carretara_o_vial  29 non-null     object 
 3   pk                29 non-null     object 
 4   sentido           29 non-null     object 
 5   tipo              29 non-null     object 
 6   longitud          29 non-null     float64
 7   latitud           29 non-null     float64
dtypes: float64(2), int64(1), object(5)
memory usage: 1.9+ KB
None


In [112]:
ds['radares'].to_csv('limpios/radares.csv', index=False)

#### Iluminación

In [113]:
get_info('iluminacion')
ds['iluminacion']

tipo_bloque        0
tipo               0
via_clase          0
via_par        12196
via_nombre         0
clase_app          0
numero             0
cod_ndp            0
distrito           0
barrio             0
x_utm              0
y_utm              0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 235466 entries, 0 to 3613
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   tipo_bloque  235466 non-null  object 
 1   tipo         235466 non-null  object 
 2   via_clase    235466 non-null  object 
 3   via_par      223270 non-null  object 
 4   via_nombre   235466 non-null  object 
 5   clase_app    235466 non-null  object 
 6   numero       235466 non-null  object 
 7   cod_ndp      235466 non-null  object 
 8   distrito     235466 non-null  int64  
 9   barrio       235466 non-null  int64  
 10  x_utm        235466 non-null  float64
 11  y_utm        235466 non-null  object 
dtypes: float64(1), int64(2),

,tipo_bloque,tipo,via_clase,via_par,via_nombre,clase_app,numero,cod_ndp,distrito,barrio,x_utm,y_utm
0,LBE002,DESCARGA,CALLE,DE LAS,ERAS,NUMERO,3,20040016,16,4,445648.4812,4480670.67
1,LBE002,DESCARGA,CALLE,DE LAS,ERAS,NUMERO,10,31024246,16,4,445610.6416,4480740.996
2,GLED001,LED,PLAZA,DE,MAR DEL PLATA,NUMERO,12,11119324,16,4,445680.0750,4480679.164
3,FFLED005,LED,CALLE,DEL,MAR AMARILLO,NUMERO,21,11119327,16,4,445669.5353,4480644.66
4,FFLED005,LED,CALLE,DEL,MAR AMARILLO,NUMERO,19,11119326,16,4,445651.1627,4480635.018
...,...,...,...,...,...,...,...,...,...,...,...,...
3609,B - Báculo.,SA - Sodio alta presión.,AUTOVIA,NaN,M-500,KM,99+370,99WT37ALUM01,9,1,437437.9738,"4476224,266,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
3610,B - Báculo.,SA - Sodio alta presión.,AUTOVIA,NaN,M-500,KM,99+420,99WT42ALUM01,9,1,437493.3321,"4476217,74,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
3611,B - Báculo.,SA - Sodio alta presión.,AUTOVIA,NaN,M-500,KM,99+460,99WT46ALUM01,9,1,437532.6933,"4476210,583,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
3612,B - Báculo.,SA - Sodio alta presión.,AUTOVIA,NaN,M-500,KM,99+500,99WT50ALUM01,9,1,437571.4231,"4476206,794,,,,,,,,,,,,,,,,,,,,,,,,,,,,"


Limpiamos las columnas de coordenadas para poder ajustar el tipo y hacer la transformación a latitud longitud

In [114]:
def clean_y_utm(x):
    x = str(x)
    if x.endswith(',' * 28):
        x = x[:-28]
    return x.replace(',', '.')

ds['iluminacion']['y_utm'] = ds['iluminacion']['y_utm'].apply(clean_y_utm)

# Convertimos las coordenadas UTM a float
ds['iluminacion']['y_utm'] = ds['iluminacion']['y_utm'].astype(float)


In [115]:
in_proj = Proj(init='epsg:25830')  # EPSG: 25830 = 30N
out_proj = Proj(init='epsg:4326')  # sistema WGS84 delatlong

ds['iluminacion']['longitud'], ds['iluminacion']['latitud'] = transform(in_proj, out_proj, ds['iluminacion']['x_utm'].values, ds['iluminacion']['y_utm'].values)


c:\Users\paula\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\paula\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\paula\AppData\Local\Temp\ipykernel_19560\1396253833.py:4: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotc

In [116]:
ds['iluminacion'] = ds['iluminacion'].drop(columns=['x_utm', 'y_utm', 'cod_ndp', 'tipo_bloque'])

Unificamos toda la dirección en un único atributo y eliminamos todas aquellas columnas redundantes o innecesarias.

In [117]:
ds['iluminacion']['via_par'] = ds['iluminacion']['via_par'].fillna('')
ds['iluminacion']['direccion'] = ds['iluminacion']['via_clase'] + " " + ds['iluminacion']['via_par'] + " " + ds['iluminacion']['via_nombre'] + " " + ds['iluminacion']['clase_app'] + " " + ds['iluminacion']['numero'].astype(str)
ds['iluminacion'] = ds['iluminacion'].drop(columns=['via_par', 'via_nombre', 'clase_app', 'numero'])

In [118]:
ds['iluminacion'].to_csv('limpios/iluminacion.csv', index=False)

#### Padrón

In [119]:
get_info('padron')
ds['padron']

ejericio                   0
cod_tipo_persona           0
tipo_persona               0
cod_distrito               0
distrito                   0
cod_barrio                 0
barrio                     0
cod_tipo_vehículo          0
tipo_vehículo              0
etiqueta_medioambiental    0
clasificación_ambiental    0
cuota                      0
tipo_carburante            0
año_matriculación          0
contador                   0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180345 entries, 0 to 180344
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   ejericio                 180345 non-null  int64 
 1   cod_tipo_persona         180345 non-null  object
 2   tipo_persona             180345 non-null  object
 3   cod_distrito             180345 non-null  int64 
 4   distrito                 180345 non-null  object
 5   cod_barrio               180345 non-null  int64 
 6   barr

,ejericio,cod_tipo_persona,tipo_persona,cod_distrito,distrito,cod_barrio,barrio,cod_tipo_vehículo,tipo_vehículo,etiqueta_medioambiental,clasificación_ambiental,cuota,tipo_carburante,año_matriculación,contador
0,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,--,Sin clasificación ambiental,73,Tipo carburante desconocido,2013,1
1,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,--,Sin clasificación ambiental,73,DIESEL,2006,1
2,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,--,Sin clasificación ambiental,73,DIESEL,2007,1
3,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,--,Sin clasificación ambiental,73,DIESEL,2009,1
4,2022,E,Ente sin Personalidad Art 35.4,0,--,0,--,CA,CAMION,--,Sin clasificación ambiental,73,DIESEL,2015,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180340,2022,J,Jurídica,21,BARAJAS,215,CORRALEJOS,TU,TURISMO,,Sin distintivo Ambiental,129,DIESEL,2001,1
180341,2022,J,Jurídica,21,BARAJAS,215,CORRALEJOS,TU,TURISMO,,Sin distintivo Ambiental,129,DIESEL,2003,1
180342,2022,J,Jurídica,21,BARAJAS,215,CORRALEJOS,TU,TURISMO,,Sin distintivo Ambiental,129,DIESEL,2005,1
180343,2022,J,Jurídica,21,BARAJAS,215,CORRALEJOS,TU,TURISMO,,Sin distintivo Ambiental,129,GASOLINA,2000,1


In [120]:
ds['padron'].rename(columns={'ejericio':'año'}, inplace=True)

In [121]:
for column in ds['padron'].columns:
    print(column)
    print(ds['padron'][column].unique())

año
[2022]
cod_tipo_persona
['E' 'F' 'J' 'O']
tipo_persona
['Ente sin Personalidad Art 35.4' 'Física' 'Jurídica' 'Otros Entes']
cod_distrito
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
distrito
['--' 'CENTRO' 'ARGANZUELA' 'RETIRO' 'SALAMANCA' 'CHAMARTIN' 'TETUAN'
 'CHAMBERI' 'FUENCARRAL - EL PARDO' 'MONCLOA - ARAVACA' 'LATINA'
 'CARABANCHEL' 'USERA' 'PUENTE DE VALLECAS' 'MORATALAZ' 'CIUDAD LINEAL'
 'HORTALEZA' 'VILLAVERDE' 'VILLA DE VALLECAS' 'VICALVARO'
 'SAN BLAS - CANILLEJAS' 'BARAJAS']
cod_barrio
[  0  11  12  13  14  15  16  21  22  23  24  25  26  27  31  32  33  34
  35  36  41  42  43  44  45  46  51  52  53  54  55  56  61  62  63  64
  65  66  71  72  73  74  75  76  81  82  83  84  85  86  87  88  91  92
  93  94  95  96  97 101 102 103 104 105 106 107 111 112 113 114 115 116
 117 121 122 123 124 125 126 127 131 132 133 134 135 136 141 142 143 144
 145 146 151 152 153 154 155 156 157 158 159 161 162 163 164 165 166 171
 172 173 174 175 181 182 183 191

Eliminamos todas las columnas que no vamos a utilizar

In [122]:
ds['padron'] = ds['padron'].drop(columns=['tipo_persona', 'cod_tipo_persona', 'cod_tipo_vehículo', 'contador', 'cuota', 'etiqueta_medioambiental', 'distrito', 'barrio', 'tipo_carburante', 'año_matriculación'])

In [123]:
ds['padron']['clasificación_ambiental'] = ds['padron']['clasificación_ambiental'].str.replace('Sin distintivo Ambiental', 'Sin clasificación ambiental')
ds['padron']['cod_distrito'] = ds['padron']['cod_distrito'].replace(0, 22)

In [124]:
ds['padron']['tipo_vehículo'] = ds['padron']['tipo_vehículo'].astype('category')
ds['padron']['clasificación_ambiental'] = ds['padron']['clasificación_ambiental'].astype('category')

In [125]:
get_info('padron')

año                        0
cod_distrito               0
cod_barrio                 0
tipo_vehículo              0
clasificación_ambiental    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180345 entries, 0 to 180344
Data columns (total 5 columns):
 #   Column                   Non-Null Count   Dtype   
---  ------                   --------------   -----   
 0   año                      180345 non-null  int64   
 1   cod_distrito             180345 non-null  int64   
 2   cod_barrio               180345 non-null  int64   
 3   tipo_vehículo            180345 non-null  category
 4   clasificación_ambiental  180345 non-null  category
dtypes: category(2), int64(3)
memory usage: 4.5 MB
None


In [126]:
ds['padron'].to_csv('limpios/padron.csv', index=False)

In [127]:
with open('diccionarios/accidenteDict.json', 'w') as file:
    json.dump(accidenteDict, file)
with open('diccionarios/distritoDict.json', 'w') as file:
    json.dump(distritoDict, file)
with open('diccionarios/lesividadDict.json', 'w') as file:
    json.dump(lesividadDict, file)